In [1]:
import re

In [2]:
rules = [
    #པ་ toujours rattaché au mot précédent
    (r' (པའ?.?་)', r'\1'),
    #ཞེ་ན་ toujours séparé
    (r'(ཞེ|ཅེ)་ན', r'\1 ་ན'),
    #གང་དག་ toujours ensemble
    (r'གང་ དག', 'གང་དག'),
    #གང་  ཅི་ + ཞིག་ idem
    (r'(གང་|ཅི་|རེ་|སུ་) ཞིག', r'\1ཞིག'),
    #གཞན་དག་ idem
    (r'གཞན་ དག', r'གཞན་དག'),
    #གང་ གཞན་  དེ་ +  ཡང་ toujours séparé
    (r'(གང་|གཞན་|དེ་)ཡང་', r'\1 ཡང་'),
    #གང་ ཡིན toujours séparé
    (r'གང་ཡིན', r'གང་ ཡིན'),
    #ལ་སོགས་པ་ toujours ensemble (devenu complètement opaque)
    (r'ལ་ ?སོགས་ ?པ', r'ལ་སོགས་པ'),
    #པོ་ བོ་ toujours ensemble (c’est de la morphologie, pas de la syntaxe)
    (r'\s(པོ|བོ)', r'\1'),
    #པ་ བ་ ensemble
    (r'\s(པ|བ)', r'\1'),
    #མཁན་ ཅན་ ཉིད་séparé, mais à décider
    (r'([^་ ]་)(མཁན|ཅན|ཉིད)', r'\1 \2'),
    #ཅིག séparé
    (r'([^་ཁ ]་)ཅིག', r'\1 ཅིག'),
    #ཕུན་སུམ་ together
    (r'ཕུན་ སུམ', r'ཕུན་སུམ'),
    #discuss about constructions such as རྔ་བོ་ཆེ་ or སྟོབས་པོ་ཆེ་ (separated for the moment)
    (r' (བོ་ ?ཆེ|པོ་ ?ཆེ)', r'\1'),
    #དེ་ … + བཞིན་ always separated
    (r'([^་ ]+་)བཞིན', r'\1 བཞིན'),
    # exception
    ('རང་ བཞིན་', 'རང་བཞིན་'),
    #… + ཙམ་ always separated
    (r'([^་ ]+་)ཙམ', r'\1 ཙམ'),
    #ཇི་ལྟ་བུ་ together
    (r'ཇི་ (ལྟ་བུ|ལྟར|སྙེད|སྐད|སྲིད)', r'ཇི་\1'),
    #བདག་ཉིད་  རང་ཉིད་ ensemble
    (r'(བདག་|རང་) ཉིད', r'\1ཉིད'),
    #དེ་བཞིན་གཤེགས་པ
    (r'དེ་ ?བཞིན་ ?གཤེགས་ ?པ', r'དེ་བཞིན་གཤེགས་པ'),
    # without པ
    (r'དེ་བཞིན་གཤེགས་([^པ])', r'དེ་བཞིན་ གཤེགས་\1'),
    (r'བདེ་ ?བ ?ར་ ?གཤེགས་ ?པ', r'བདེ་བར་གཤེགས་པ'),
    # without པ
    (r'དེ་བར་གཤེགས་([^པ])', r'དེ་བར་ གཤེགས་\1'),
    #ཡང་དག་པར་ together except particle
    (r'ཡང་ ?དག་ ?པ ?ར', r'ཡང་དག་པ ར'),
    #བདག་ཉིད་ together
    (r'བདག་ ?ཉིད', r'བདག་ཉིད'),
    #ཉམ་ང་བ together
    (r'ཉམ་ ?ང་ ?བ', r'ཉམ་ང་བ'),
    #བཅོམ་ལྡན་འདས་ together
    (r'བཅོམ་ ?ལྡན་ ?འདས', r'བཅོམ་ལྡན་འདས'),
    #ཇི་སྙེད་པ together
    (r'ཇི་ ?སྙེད་ ?པ', r'ཇི་སྙེད་པ'),
    #དེ་ཁོ་ན་ཉིད་ together
    (r'དེ་ ?ཁོ་ ?ན་ ?ཉིད', r'དེ་ཁོ་ན་ཉིད'),
    #མཐའ་དག་ together
    (r'མཐའ་ ?དག', r'མཐའ་དག'),
    #དེ་བཞིན་ཉིད་ together
    (r'དེ་ ?བཞིན་ ?ཉིད', r'དེ་བཞིན་ཉིད'),
    #དེ་བས་ན་ no idea, left together
    (r'དེ་ ?བ ?ས་ ?ན', r'དེ་བས་ན'),
    #ཡང་དག་པ together
    (r'ཡང་ ?དག་ ?པ', r'ཡང་དག་པ'),
    #དེ་ལྟ་བུ no idea, left together
    (r'དེ་ ?ལྟ་ ?བུ', r'དེ་ལྟ་བུ'),
    #དེ་ལྟར་ idem
    (r'དེ་ ?ལྟ ?ར', r'དེ་ལྟར'),
    #ཅི་ཞིག་ together
    (r'ཅི་ ?ཞིག', r'ཅི་ཞིག'),
    #དེ ས་ separated
    (r'དེས', r'དེ ས'),
    #གང་ ཡིན་པ་ separated
    (r'གང་ ?ཡིན་ ?པ', r'གང་ ཡིན་པ་'),
    #ཕལ་ཆེ་བ left together
    (r'ཕལ་ ?ཆེ་ ?བ', r'ཕལ་ཆེ་བ'),
    #ཕྱི་རོལ་ together
    (r'ཕྱི་ ?རོལ', r'ཕྱི་རོལ'),
    #ཕྱིན་ཅི་ མ་ ལོག་པ no idea, but left like that to accomodate the affirmative and negative form
    (r'ཕྱིན་ ?ཅི་ ?མ་ ?ལོག་ ?པ', r'ཕྱིན་ཅི་ མ་ ལོག་པ'),
    #རྒྱ་ ཆེ ར་ no idea but separated
    (r'རྒྱ་ ?ཆེ ?ར་', r'རྒྱ་ ཆེ ར་'),
    #མི་མཐུན་ཕྱོགས་ together
    (r'མི་ ?མཐུན་ ?ཕྱོགས', r'མི་མཐུན་ཕྱོགས'),
    #འདི་ལྟར་ no idea, left as is
    (r'འདི་ ?ལྟ ?ར', r'འདི་ལྟར'),
    #ཇི་བཞིན་ no idea, left together
    (r'ཇི་ ?བཞིན', r'ཇི་བཞིན'),
    #རྣམ་ དག་ separated to hint the missing parts
    (r'རྣམ་དག', r'རྣམ་ དག'),
    #ལྷ་མ་ཡིན་ together
    (r'ལྷ་ ?མ་ ?ཡིན', r'ལྷ་མ་ཡིན'),
    #ཐབས་ཚུལ་ ???
    (r'ཐབས་ཚུལ', r'ཐབས་ ཚུལ'),
    #ཉེ ར་ ཞི་ separated
    (r'ཉེ ?ར་ ?ཞི', r'ཉེ ར་ ཞི'),
    #ཆོམ་ རྐུན་ (separated as for now)
    (r'ཆོམ་རྐུན', r'ཆོམ་ རྐུན'),
    #ཆོས་ སྒྲ་ (idem)
    (r'ཆོས་སྒྲ', r'ཆོས་ སྒྲ'),
    #བདེ་ གཤེགས་ separated to hint there are missing syls
    (r'བདེ་གཤེགས', r'བདེ་ གཤེགས'),
    #དཔག་ མེད་ idem
    (r'དཔག་མེད', r'དཔག་ མེད'),
    #དེ་ ཉིད་ depends
    (r'དེ་ཉིད', r'དེ་ ཉིད'),
    #དེ་ནས་ depends but most likely is in two, since in literal Tibetan
    (r'དེ་ནས', r'དེ་ ནས'),
]


In [3]:
ambiguous = [
    (r'ཀུན་ ?མཁྱེན་', r'ཀུན་\?མཁྱེན་'), 
    (r'གྲངས་ ?མེད་', r'གྲངས་\?མེད་'),
    (r'ཐུབ་ ?དབང་', r'ཐུབ་\?དབང་'),
    (r'དབུ་ ?མ་', r'དབུ་\?མ་'),
    (r'དེ་ ?བཞིན་ ?གཤེགས་པ', r'དེ་\?བཞིན་\?གཤེགས་པ'),
    (r'ཡང་ ?དག་([^པ])', r'ཡང་\?དག་\1'),
    (r'དོན་ ?དམ་པ', r'དོན་\?དམ་པ'),
    (r'བདེ་ ?ལེགས་', r'བདེ་\?ལེགས་'),
    (r'མ་ ?ལུས་པ', r'མ་\?ལུས་པ'),
    (r'མཆོད་ ?རྟེན་', r'མཆོད་\?རྟེན་'),
    (r'མཆོད་ ?སྦྱིན་', r'མཆོད་\?སྦྱིན་'),
    (r'མི་ ?རྟག་', r'མི་\?རྟག་'),
    (r'རྣམ་ ?རྟོག་', r'རྣམ་\?རྟོག་'),
    (r'གནས་ ?མ་', r'གནས་\?མ་'),
    (r'གཞན་ ?དག་', 'གཞན་\?དག་'),
    (r'བདག་ ?གཞན་ ?དོན་', r'བདག་\?གཞན་\?དོན་'),
    (r'མཚན་ ?ཉིད་', r'མཚན་\?ཉིད་'),
    (r'ལྷ་ ?མིན་', r'ལྷ་\?མིན་'),
    (r'སྨིག་ ?རྒྱུ་', r'སྨིག་\?རྒྱུ་')
]


In [4]:
corrections = [
    ('ཀུན་ རྟོག་ ཐམས་ཅད་ སྤང་པ', 'ཀུན་རྟོག་ ཐམས་ཅད་ སྤང་པ'),
    ('ཀུན་ལ ས་', 'ཀུན་ ལས་'),
    ('ཀྱི་ ཧུད་', 'ཀྱི་ཧུད་'),
    ('ཀྱི འོ', 'ཀྱིའོ'),
    ('ཀྱེ་མ འོ་', 'ཀྱེ་མའོ་'),
    ('ཀྲྀཥྞ་པཎྜི་ ཏ་', 'ཀྲྀཥྞ་པཎྜི་ཏ་'),
    ('ཁ་ ཅིག་', 'ཁ་ཅིག་'),
    ('ཁ་ ཆེའི་པཎྜི་ ཏ་', 'ཁ་ཆེ འི་ པཎྜི་ཏ་'),
    ('ཁ་ ལོས་', 'ཁ་ལོ ས་'),
    ('ཁམས་གསུམ་ རྒྱུ་ དང་ མི་ རྒྱུ་བ', 'ཁམས་ གསུམ་ རྒྱུ་ དང་ མི་ རྒྱུ་བ'),
    ('ཁམས་གསུམ་པ་ ཐམས་ཅད་ ཀྱི་ དབང་ཕྱུག་ཆེན་པོ', 'ཁམས་ གསུམ་པ་ ཐམས་ཅད་ ཀྱི་ དབང་ཕྱུག་ ཆེན་པོ'),
    ('ཁོང་སྟོང་', 'ཁོང་ སྟོང་'),
    ('ཁོང་ཁྲོ་ བར', 'ཁོང་ཁྲོ་བ ར'),
    ('ཁྱད་མེད་', 'ཁྱད་ མེད་'),
    ('ཁྱད་པར་གནས་', 'ཁྱད་པར་ གནས་'),
    ('ཁྲོན་ རྙིང་', 'ཁྲོན་རྙིང་'),
    ('གང་ དག་ མཉམ་པ ས་ དཔེ་ ཡིན་པ་ དེ་དག་ ནི་ མཉམ་པ འི་ དཔེའོ', 'གང་དག་ མཉམ་པ ས་ དཔེ་ ཡིན་པ་ དེ་དག་ ནི་ མཉམ་པ འི་ དཔེ འོ'),
    ('གང་དང་གང་ དག་', 'གང་ དང་ གང་དག་'),
    ('གང་དེ་', 'གང་ དེ་'),
    ('གང་ཞིག་ རྗེས་ སུ་ མི་མཐུན་པ', 'གང་ ཞིག་ རྗེས་ སུ་ མི་ མཐུན་པ'),
    ('གང་ཞིག་ པ', 'གང་ ཞིག་པ'),
    ('གང་ཡོད', 'གང་ ཡོད'),
    ('གཅིག་ཉིད་', 'གཅིག་ ཉིད་'),
    ('གཅིག་ལ་གཅིག་', 'གཅིག་ ལ་ གཅིག་'),
    ('གཉི་ ག་ལ ས་', 'གཉི་ག་ ལས་'),
    ('གཉིས་ ཀ', 'གཉིས་ཀ'),
    ('གཉིས་ ཀ་', 'གཉིས་ཀ་'),
    ('གཉིས་ ཀ་ མ་ སྐྱེས་པར', 'གཉིས་ཀ་ མ་ སྐྱེས་པ ར'),
    ('གཉིས་ ཀར་', 'གཉིས་ཀ ར་'),
    ('གཏིང་ མཐའ་མེད་པ', 'གཏིང་མཐའ་ མེད་པ'),
    ('གཏོང་ བ་', 'གཏོང་བ་'),
    ('གདུལ་ བྱ', 'གདུལ་བྱ'),
    ('གདུལ་བྱ འི་ འགྲོ་བ་ མ་ལུས་པ', 'གདུལ་ བྱ འི་ འགྲོ་བ་ མ་ ལུས་པ'),
    ('གདོད་ མ་', 'གདོད་མ་'),
    ('གནག་ རྗེས་', 'གནག་རྗེས་'),
    ('གནས་ཆེན་ པོ་', 'གནས་ ཆེན་པོ་'),
    ('གནས་མིན་', 'གནས་ མིན་'),
    ('གནས་འགྱུ ར', 'གནས་ འགྱུར'),
    ('གནས་ སྐབས་', 'གནས་སྐབས་'),
    ('གཙང་ བ་', 'གཙང་བ་'),
    ('གཙོ་ བོ་', 'གཙོ་བོ་'),
    ('གཙོ་ བོས་', 'གཙོ་བོ ས་'),
    ('གཙོར་', 'གཙོ ར་'),
    ('གཞན་ དོན་', 'གཞན་དོན་'),
    ('གཞན་ དོན་ ལ་ ཆགས་ ཐུགས་ མངའ་ བས', 'གཞན་དོན་ ལ་ ཆགས་ ཐུགས་ མངའ་བ ས'),
    ('གཞན་ དོན་གྲུབ་པར་', 'གཞན་དོན་ གྲུབ་པ ར་'),
    ('གཞལ་ བར་', 'གཞལ་བ ར་'),
    ('གཞལ་ཡས་ ཁང་', 'གཞལ་ཡས་ཁང་'),
    ('གཞི་མེད་ པ་', 'གཞི་ མེད་པ་'),
    ('གཞོན་ ནུ', 'གཞོན་ནུ'),
    ('གཞོན་ ནུར་', 'གཞོན་ནུ ར་'),
    ('གཞོན་ ནུར་ གྱུར་པ་ ལ་ ཕྱག་འཚལ་', 'གཞོན་ནུ ར་ གྱུར་པ་ ལ་ ཕྱག་ འཚལ་'),
    ('གཞོལ་ བའོ', 'གཞོལ་བ འོ'),
    ('གཟིགས་པ་ པོ', 'གཟིགས་པ་པོ'),
    ('གཟིགས་པར', 'གཟིགས་པ ར'),
    ('གཟིར་ བར་', 'གཟིར་བ ར་'),
    ('གཟུགས་ བརྙན་', 'གཟུགས་བརྙན་'),
    ('གཟུགས་ སྐུ་', 'གཟུགས་སྐུ་'),
    ('གཟུགས་ཅན་སྙིང་པོ འི་ གླང་ཆེན་སྨྱོན་པ་', 'གཟུགས་ཅན་ སྙིང་པོ འི་ གླང་ཆེན་ སྨྱོན་པ་'),
    ('གཟུང་ བ་', 'གཟུང་བ་'),
    ('གཟོད་ནས་', 'གཟོད་ ནས་'),
    ('གཡང་ ས', 'གཡང་ས'),
    ('གཡེང་ བ', 'གཡེང་བ'),
    ('གཡོ་ བ་', 'གཡོ་བ་'),
    ('གཡོ་ བ་ ནི་ གཞན་ དུ་ འགྱུར་ བའོ', 'གཡོ་བ་ ནི་ གཞན་ དུ་ འགྱུར་བ འོ'),
    ('གཡོ་ བ་ མེད་པ ས་ ནི་ མ་ གཡོ་ བ་', 'གཡོ་བ་ མེད་པ ས་ ནི་ མ་ གཡོ་བ་'),
    ('གཡོས་པའོ', 'གཡོས་པ འོ'),
    ('གར་མཁན་', 'གར་ མཁན་'),
    ('གལ་ ཏེ་', 'གལ་ཏེ་'),
    ('གལ་ ཏེ་ སེམས་ ཤིག་ ཡོད་ ན་ནི', 'གལ་ཏེ་ སེམས་ ཤིག་ ཡོད་ ན་ ནི'),
    ('གསལ་ བར་', 'གསལ་བ ར་'),
    ('གསེར་བཟང་དྲི་མེད་', 'གསེར་བཟང་ དྲི་མེད་'),
    ('གསོ་ བ་', 'གསོ་བ་'),
    ('གསོལ་ པ་', 'གསོལ་པ་'),
    ('གི་པདྨ་', 'གི་ པདྨ་'),
    ('གོས་མེད་', 'གོས་ མེད་'),
    ('གྱུར་ཅིག', 'གྱུར་ ཅིག'),
    ('གྱུར་པས', 'གྱུར་པ ས'),
    ('གྲུབ་མཆོག་', 'གྲུབ་ མཆོག་'),
    ('གྲོང་ ཁྱེར་', 'གྲོང་ཁྱེར་'),
    ('གྲོལ་ བ་', 'གྲོལ་བ་'),
    ('གླེན་གང་', 'གླེན་ གང་'),
    ('གློ་བ་དགའ་', 'གློ་བ་ དགའ་'),
    ('ང་ རྒྱལ་', 'ང་རྒྱལ་'),
    ('ངག་གི་སྤྱོད་ཡུལ་', 'ངག་ གི་ སྤྱོད་ཡུལ་'),
    ('ངན་ འགྲོས་', 'ངན་འགྲོ ས་'),
    ('ངན་ སོང་', 'ངན་སོང་'),
    ('ངན་འགྲོའི་སྡུག་ བསྔལ་', 'ངན་འགྲོ འི་ སྡུག་བསྔལ་'),
    ('ངེས་ འབྱུང་', 'ངེས་འབྱུང་'),
    ('ངེས་པའི་དོན་', 'ངེས་པ འི་ དོན་'),
    ('ངེས་པར་ལེགས་པ', 'ངེས་པ ར་ ལེགས་པ'),
    ('ངེས་པར་', 'ངེས་པ ར་'),
    ('ངེས་བསྟེན་', 'ངེས་ བསྟེན་'),
    ('ངོ་ བོ་', 'ངོ་བོ་'),
    ('ངོ་བོ་ ཉིད', 'ངོ་བོ་ཉིད'),
    ('ཅིའི་ཕྱིར་', 'ཅི འི་ ཕྱིར་'),
    ('ཅི་ཕྱིར་', 'ཅི་ ཕྱིར་'),
    ('ཅི་ཡང་མེད་པ', 'ཅི་ ཡང་ མེད་པ'),
    ('ཅུང་ ཟད་', 'ཅུང་ཟད་'),
    ('ཅེ་ན', 'ཅེ་ ན'),
    ('ཅེས་བྱ་བ་', 'ཅེས་ བྱ་བ་'),
    ('ཆ་མཐུན་པ', 'ཆ་ མཐུན་པ'),
    ('ཆགས་པའི', 'ཆགས་པ འི'),
    ('ཆགས་པར་ གྱུར་པ་ མི་ མངའ་ བས', 'ཆགས་པ ར་ གྱུར་པ་ མི་ མངའ་བ ས'),
    ('ཆགས་པས', 'ཆགས་པ ས'),
    ('ཆུ་ ཤིང་', 'ཆུ་ཤིང་'),
    ('ཆུ་ སྲིན་ གྱིས་ ནི་ ལུས་ ཟོས་པས', 'ཆུ་སྲིན་ གྱིས་ ནི་ ལུས་ ཟོས་པ ས'),
    ('ཆུང་ མ་དག་', 'ཆུང་མ་ དག་'),
    ('ཆེར་', 'ཆེ ར་'),
    ('ཆེར་ བཅས་པའི', 'ཆེ ར་ བཅས་པ འི'),
    ('ཆོས་ཀྱི་མཆོག་', 'ཆོས་ ཀྱི་ མཆོག་'),
    ('ཆོས་ཀྱི་འཁོར་ལོ་', 'ཆོས་ ཀྱི་ འཁོར་ལོ་'),
    ('ཆོས་ཀྱི་རང་བཞིན་', 'ཆོས་ ཀྱི་ རང་བཞིན་'),
    ('ཆོས་ཀྱི་སྣོད་ གྱུར་ཅིག', 'ཆོས་ ཀྱི་ སྣོད་ གྱུར་ ཅིག'),
    ('ཆོས་ ཉིད་', 'ཆོས་ཉིད་'),
    ('ཆོས་ དབྱིངས་', 'ཆོས་དབྱིངས་'),
    ('ཆོས་ སྐུ་', 'ཆོས་སྐུ་'),
    ('ཆོས་སྟོན་པ་', 'ཆོས་ སྟོན་པ་'),
    ('ཆོས་སྦྱིན་ པ', 'ཆོས་ སྦྱིན་པ'),
    ('ཇི་ ལྟ་ བུ', 'ཇི་ལྟ་བུ'),
    ('ཇི་ ལྟ་ བུར', 'ཇི་ལྟ་བུ ར'),
    ('ཇི་ ལྟར་ མར་ངོ འི་ བཅུ་ བཞི་', 'ཇི་ལྟར་ མར་ངོ འི་ བཅུ་བཞི་'),
    ('ཇི་ ལྟར་ རྗེས་སུ་', 'ཇི་ལྟར་ རྗེས་ སུ་'),
    ('ཇི་ སྐད་ དུ་ དེ་བཞིན་ གཤེགས་པ', 'ཇི་སྐད་ དུ་ དེ་བཞིན་གཤེགས་པ'),
    ('ཇི་ལྟར་འཇུག་ པར་', 'ཇི་ལྟར་ འཇུག་པ ར་'),
    ('ཇི་ལྟར་ཡང༌', 'ཇི་ལྟར་ ཡང༌'),
    ('ཉན་ ཐོས་', 'ཉན་ཐོས་'),
    ('ཉན་ ཐོས་ དང་ ནི་ རང་ སངས་ རྒྱས', 'ཉན་ཐོས་ དང་ ནི་ རང་སངས་རྒྱས'),
    ('ཉནཐོས་ཀྱིས་', 'ཉནཐོས་ ཀྱིས་'),
    ('ཉམ་ པ་', 'ཉམ་པ་'),
    ('ཉམས་པར་', 'ཉམས་པ ར་'),
    ('ཉམས་པས', 'ཉམས་པ ས'),
    ('ཉི་ མ', 'ཉི་མ'),
    ('ཉི་མའི་རིགས་', 'ཉི་མ འི་ རིགས་'),
    ('ཉེ་ བར་', 'ཉེ་བ ར་'),
    ('ཉེ་ བར་གནས་', 'ཉེ་བ ར་ གནས་'),
    ('ཉེ་བར་ལེན་པ', 'ཉེ་བ ར་ ལེན་པ'),
    ('ཉེས་བཤད་', 'ཉེས་ བཤད་'),
    ('ཉེས་མེད་', 'ཉེས་ མེད་'),
    ('ཉོན་ མོངས', 'ཉོན་མོངས'),
    ('ཉོན་ མོངས་', 'ཉོན་མོངས་'),
    ('ཉོན་མོངས་ཅན་', 'ཉོན་མོངས་ ཅན་'),
    ('ཉོན་མོངས་པས', 'ཉོན་མོངས་པ ས'),
    ('ཉོནམོངས་རྣམས་', 'ཉོནམོངས་ རྣམས་'),
    ('ཏིང་ ངེ་ འཛིན་', 'ཏིང་ངེ་འཛིན་'),
    ('ཐ་ དད་པ', 'ཐ་དད་པ'),
    ('ཐ་ མས་', 'ཐ་མ ས་'),
    ('ཐ་ སྙད་', 'ཐ་སྙད་'),
    ('ཐ་ དད་', 'ཐ་དད་'),
    ('ཐབས་མཁས་', 'ཐབས་ མཁས་'),
    ('ཐམས་ ཅད་', 'ཐམས་ཅད་'),
    ('ཐམས་ཅད་མཁྱེན་པ་', 'ཐམས་ཅད་ མཁྱེན་པ་'),
    ('ཐམས་ཅད་སྟོང་པ་', 'ཐམས་ཅད་ སྟོང་པ་'),
    ('ཐམས་ཅད་ པ ས་ ཕུལ་ དུ་གྱུར་པ', 'ཐམས་ཅད་པ ས་ ཕུལ་ དུ་ གྱུར་པ'),
    ('ཐུགས་བརྩེ་བ ས', 'ཐུགས་ བརྩེ་ བས'),
    ('ཐུགས་ རྗེ་', 'ཐུགས་རྗེ་'),
    ('ཐུགས་ རྗེ་བཙུན་ མོས་', 'ཐུགས་རྗེ་ བཙུན་མོ ས་'),
    ('ཐུགས་རྗེའི་བདག', 'ཐུགས་རྗེ འི་ བདག'),
    ('ཐུགས་རྗེའི་བདག་ ཉིད་', 'ཐུགས་རྗེ འི་ བདག་ཉིད་'),
    ('ཐུགས་རྗེའི་བདག་ ཉིད་ ངོ་བོ་ཉིད་ ཡོད་པ་ དེ་དག་ ནི་ ཐུགས་རྗེའི་བདག་ ཉིད་', 'ཐུགས་རྗེ འི་ བདག་ཉིད་ ངོ་བོ་ ཉིད་ ཡོད་པ་ དེ་དག་ ནི་ ཐུགས་རྗེ འི་ བདག་ཉིད་'),
    ('ཐུགས་རྗེ་ཆེ་', 'ཐུགས་རྗེ་ ཆེ་'),
    ('ཐུགས་རྗེ་ཆེ་ དང་ལྡན་པ་ པོ', 'ཐུགས་རྗེ་ ཆེ་ དང་ ལྡན་པ་པོ'),
    ('ཐུགས་རྗེ་ཆེན་པོ་', 'ཐུགས་རྗེ་ ཆེན་པོ་'),
    ('ཐུབ་ཆེན་', 'ཐུབ་ ཆེན་'),
    ('ཐུབ་པ་ཆེན་པོ་', 'ཐུབ་པ་ ཆེན་པོ་'),
    ('ཐེག་པ་གསུམ་', 'ཐེག་པ་ གསུམ་'),
    ('ཐེག་པ་ཆེན་པོ འི་ ངང་ཚུལ་ དུ་གྱུར་པ་', 'ཐེག་པ་ ཆེན་པོ འི་ ངང་ཚུལ་ དུ་ གྱུར་པ་'),
    ('ཐོག་དྲུང་', 'ཐོག་ དྲུང་'),
    ('ཐོག་ མ་', 'ཐོག་མ་'),
    ('ཐོགས་པ་མེད་པ འི་ ཡེ་ཤེས་ གཟིགས་པ་ ཞེས་བྱ་བ', 'ཐོགས་པ་ མེད་པ འི་ ཡེ་ཤེས་ གཟིགས་པ་ ཞེས་ བྱ་བ'),
    ('ད་ནི་', 'ད་ ནི་'),
    ('ད་ལྟར་', 'ད་ལྟ ར་'),
    ('དཀའ་ བ་', 'དཀའ་བ་'),
    ('དཀའ་སྤྱད་', 'དཀའ་ སྤྱད་'),
    ('དཀོན་མཆོག་གསུམ་', 'དཀོན་མཆོག་ གསུམ་'),
    ('དཀོར་ནི་', 'དཀོར་ ནི་'),
    ('དཀྱིལ་ འཁོར་སྐར་ མ་', 'དཀྱིལ་འཁོར་ སྐར་མ་'),
    ('དཀྲིས་ པ', 'དཀྲིས་པ'),
    ('དཀྲུགས་ པ་', 'དཀྲུགས་པ་'),
    ('དགའ་ བ་', 'དགའ་བ་'),
    ('དགའ་ བོ་', 'དགའ་བོ་'),
    ('དགེ་ བ', 'དགེ་བ'),
    ('དགེ་ བ་', 'དགེ་བ་'),
    ('དགེ་བའི་ལས་', 'དགེ་བ འི་ ལས་'),
    ('དགོངས་པར', 'དགོངས་པ ར'),
    ('དགོན་དང་', 'དགོན་ དང་'),
    ('དགྲ་ བཅོམ་པ་', 'དགྲ་བཅོམ་པ་'),
    ('དང་ བ', 'དང་བ'),
    ('དང་ལ ས', 'དང་ ལས'),
    ('དང་ལྡན་པ', 'དང་ ལྡན་པ'),
    ('དང་ལྡན་པ་ རྣམས་ ནི་ སྦྱིན་པ་ པོ་', 'དང་ ལྡན་པ་ རྣམས་ ནི་ སྦྱིན་པ་པོ་'),
    ('དང་པོའོ', 'དང་པོ འོ'),
    ('དང་འདྲ', 'དང་ འདྲ'),
    ('དངོས་མེད་', 'དངོས་ མེད་'),
    ('དངོས་ པ་', 'དངོས་པ་'),
    ('དངོས་པོ་གཞན་', 'དངོས་པོ་ གཞན་'),
    ('དངོས་པོ་མེད་པ་', 'དངོས་པོ་ མེད་པ་'),
    ('དངོས་མ་', 'དངོས་ མ་'),
    ('དཔག་ཏུ་མེད་པ', 'དཔག་ ཏུ་ མེད་པ'),
    ('དཔག་ཏུ་མེད་པ་', 'དཔག་ ཏུ་ མེད་པ་'),
    ('དཔའ་ བོ་', 'དཔའ་བོ་'),
    ('དཔེ་མེད་', 'དཔེ་ མེད་'),
    ('དཔེ་མེད་ པར་', 'དཔེ་ མེད་པ ར་'),
    ('དཔེ་མེད་ པ་', 'དཔེ་ མེད་པ་'),
    ('དཔེའོ', 'དཔེ འོ'),
    ('དཔེས', 'དཔེ ས'),
    ('དབང་བསྐུར་ བ་', 'དབང་ བསྐུར་བ་'),
    ('དབང་ཕྱུག་ཆེན་པོ་', 'དབང་ཕྱུག་ ཆེན་པོ་'),
    ('དབུགས་འབྱིན་པ', 'དབུགས་ འབྱིན་པ'),
    ('དབུལ་ བ འི་ འདམ་དུ་', 'དབུལ་བ འི་ འདམ་ དུ་'),
    ('དབེན་པའི་གནས་', 'དབེན་པ འི་ གནས་'),
    ('དམ་པའི་ཆོས་', 'དམ་པ འི་ ཆོས་'),
    ('དམ ར་', 'དམར་'),
    ('དམིགས་མེད་', 'དམིགས་ མེད་'),
    ('དམུ་ རྒོད་', 'དམུ་རྒོད་'),
    ('དམྱལ་ བར་', 'དམྱལ་བ ར་'),
    ('དུ་གྱུར་པ', 'དུ་ གྱུར་པ'),
    ('དུ་གྱུར་པ་', 'དུ་ གྱུར་པ་'),
    ('དུ་ བས', 'དུ་བ ས'),
    ('དུ་ མ་', 'དུ་མ་'),
    ('དུ་ཡོད་ པར་', 'དུ་ ཡོད་པ ར་'),
    ('དུག་གསུམ་', 'དུག་ གསུམ་'),
    ('དུབ་པར་མ་', 'དུབ་པ ར་ མ་'),
    ('དུལ་ བ་', 'དུལ་བ་'),
    ('དུས་གཅིག་', 'དུས་ གཅིག་'),
    ('དུས་འདས་པ', 'དུས་ འདས་པ'),
    ('དེ་ཉིད་ཀྱི་ཕྱིར་', 'དེ་ ཉིད་ ཀྱི་ ཕྱིར་'),
    ('དེ་ཉིད་ ཀྱིས་ ན་ མི་འགྱུར་བ་ སྟེ་ རང་གི་ངོ་བོ་', 'དེ་ ཉིད་ ཀྱིས་ ན་ མི་ འགྱུར་བ་ སྟེ་ རང་ གི་ ངོ་བོ་'),
    ('དེ་ཉིད་ ཀྱིས་ ནམ་མཁའ་ ལྟར་ རོ་གཅིག་ པ་', 'དེ་ ཉིད་ ཀྱིས་ ནམ་མཁའ་ ལྟར་ རོ་ གཅིག་པ་'),
    ('དེ་ཉིད་ ཁྱོད་ ཀྱིས་ ཤིན་ཏུ་', 'དེ་ ཉིད་ ཁྱོད་ ཀྱིས་ ཤིན་ ཏུ་'),
    ('དེ་ཉིད་ ཐམས་ཅད་མཁྱེན་པ་', 'དེ་ ཉིད་ ཐམས་ཅད་ མཁྱེན་པ་'),
    ('དེ་ཉིད་རིག་ པ', 'དེ་ ཉིད་ རིག་པ'),
    ('དེ་ཉིད་རིག་ པ་', 'དེ་ ཉིད་ རིག་པ་'),
    ('དེ་ དག', 'དེ་དག'),
    ('དེ་ དག་', 'དེ་དག་'),
    ('དེ་ དག་ གིས་ བསྟན་པ་ ལ་ ནན་ཏན་ བྱས་པ ས་ འགྲོ་བ་ མཐའ་དག་ ལ་ ཕན་པ་ བདེ་བ ར་ གཤེགས་པ', 'དེ་དག་ གིས་ བསྟན་པ་ ལ་ ནན་ཏན་ བྱས་པ ས་ འགྲོ་བ་ མཐའ་དག་ ལ་ ཕན་པ་ བདེ་བར་གཤེགས་པ'),
    ('དེ་ དག་ སངས་རྒྱས་ འཚམ་ མིན་པས', 'དེ་དག་ སངས་རྒྱས་ འཚམ་ མིན་པ ས'),
    ('དེ་དང་དེ', 'དེ་ དང་ དེ'),
    ('དེ་དང་དེ་', 'དེ་ དང་ དེ་'),
    ('དེ་དང་དེ་ དག་', 'དེ་ དང་ དེ་དག་'),
    ('དེ་ནས་ འབད་པ་ དང་ རྩོལ་བ འི་ བརྩོན་འགྲུས་ དང་ལྡན་པ', 'དེ་ ནས་ འབད་པ་ དང་ རྩོལ་བ འི་ བརྩོན་འགྲུས་ དང་ ལྡན་པ'),
    ('དེ་ཕྱིར་', 'དེ་ ཕྱིར་'),
    ('དེ་ཕྱིར་ བདག་ ནི་ ཤིན་ཏུ་ སྐལ་དམན་', 'དེ་ ཕྱིར་ བདག་ ནི་ ཤིན་ཏུ་ སྐལ་ དམན་'),
    ('དེ་བཞིན་གཤེགས་པ འི་ སྐུ་ འདི་ནི', 'དེ་ བཞིན་ གཤེགས་པ འི་ སྐུ་ འདི་ ནི'),
    ('དེ་བཞིན་ ལས་ བདག་ གི ར་ བྱ་བ', 'དེ་ བཞིན་ ལས་ བདག་ གི ར་ བྱ་ བ'),
    ('དེ་ བས་', 'དེ་བ ས་'),
    ('དེ་ བས་ ན་ དགོས་པ་ དང་ ཚིགས་ སུ་ བཅད་པ་ དག་ རྗོད་པ་ དང་ བརྗོད་པ ར་ བྱ་བ འི་ མཚན་ཉིད་ དུ་ འབྲེལ་པའམ', 'དེ་བས་ན་ དགོས་པ་ དང་ ཚིགས་ སུ་ བཅད་པ་ དག་ རྗོད་པ་ དང་ བརྗོད་པ ར་ བྱ་བ འི་ མཚན་ཉིད་ དུ་ འབྲེལ་ པ འམ'),
    ('དེ་མིན་', 'དེ་ མིན་'),
    ('དེ་འོག་', 'དེ་ འོག་'),
    ('དེ་ཡང་ སྨྲ་ པོ', 'དེ་ ཡང་ སྨྲ་པོ'),
    ('དེ་ཡང༌', 'དེ་ ཡང༌'),
    ('དེ་ལས་', 'དེ་ ལས་'),
    ('དེ་ལས་ གསུངས་པ འི་ མཆོག་གི་དངོས་གྲུབ་', 'དེ་ ལས་ གསུངས་པ འི་ མཆོག་ གི་ དངོས་གྲུབ་'),
    ('དེ་ལྟ', 'དེ་ ལྟ'),
    ('དེ་ལྟ་ན་', 'དེ་ ལྟ་ ན་'),
    ('དེ་ ལྟ་བུ འི་ ཆོས་ ཀྱི་ སྐུ་ སྟེ་ ཆོས་ ཀྱི་ ལུས་ དེ་ ལ་ བདག་ ཕྱག་འཚལ་', 'དེ་ལྟ་བུ འི་ ཆོས་ ཀྱི་ སྐུ་ སྟེ་ ཆོས་ ཀྱི་ ལུས་ དེ་ ལ་ བདག་ ཕྱག་ འཚལ་'),
    ('དེ་ ལྟར་ ཁྱོད་ ཀྱི་ ལོངས་ སྤྱད་པ', 'དེ་ལྟར་ ཁྱོད་ ཀྱི་ ལོངས་སྤྱད་པ'),
    ('དེ་ ལྟར་ རྣལ་ འབྱོར་པ', 'དེ་ལྟར་ རྣལ་འབྱོར་པ'),
    ('དེ་ སྐད་', 'དེ་སྐད་'),
    ('དེ་ སྲིད་', 'དེ་སྲིད་'),
    ('དེ་པར', 'དེ་པ ར'),
    ('དེ་བཞིན་ གཤེགས་པ་ ཐམས་ཅད་ ལ་ ཕྱག་འཚལ་', 'དེ་བཞིན་གཤེགས་པ་ ཐམས་ཅད་ ལ་ ཕྱག་ འཚལ་'),
    ('དེ་ལྟ་ བུའོ', 'དེ་ལྟ་བུ འོ'),
    ('དེ་ལྟ་ བུར', 'དེ་ལྟ་བུ ར'),
    ('དེའི་', 'དེ འི་'),
    ('དེའི་ ཕྱིར་ འདི ར་ ཅི་ ཞིག་ འཆད་པ ར་ འགྱུར་ ཞེས་ འདི་ ལ་ ཡང་དག་པར་', 'དེ འི་ ཕྱིར་ འདི ར་ ཅི་ཞིག་ འཆད་པ ར་ འགྱུར་ ཞེས་ འདི་ ལ་ ཡང་དག་པ ར་'),
    ('དེའི་ ཕྱིར་ འདིར', 'དེ འི་ ཕྱིར་ འདི ར'),
    ('དེའི་ ཕྲ་མོ་ ནི་ ཤིན་ཏུ་ ཕྲ་བ འི་ ཆའོ', 'དེ འི་ ཕྲ་མོ་ ནི་ ཤིན་ཏུ་ ཕྲ་བ འི་ ཆ འོ'),
    ('དེའི་ འབྲས་གཟུགས་ དང་ གཟུགས་ མེད་པའང༌', 'དེ འི་ འབྲས་གཟུགས་ དང་ གཟུགས་ མེད་པ འང༌'),
    ('དེར་', 'དེ ར་'),
    ('དེར་ ནི་ མཆོག་ སངས་རྒྱས་ ཀྱི་ ཆོས་ རྣམས་ ཀྱི་ དངོས་གྲུབ་ ཡོངས་ སུ་ གྲུབ་པ་ དེ་ སྩོལ་བ་ ཞེས་ རྣམ་པ ར་ བཤད་པ ར་ བྱའོ', 'དེ ར་ ནི་ མཆོག་ སངས་རྒྱས་ ཀྱི་ ཆོས་ རྣམས་ ཀྱི་ དངོས་གྲུབ་ ཡོངས་ སུ་ གྲུབ་པ་ དེ་ སྩོལ་བ་ ཞེས་ རྣམ་པ ར་ བཤད་པ ར་ བྱ འོ'),
    ('དེས་ ནི་ དེ་བཞིན་ གཤེགས་པ་', 'དེ ས་ ནི་ དེ་བཞིན་གཤེགས་པ་'),
    ('དེས་ ནི་ སེམས་ཅན་ མ་ལུས་', 'དེ ས་ ནི་ སེམས་ཅན་ མ་ ལུས་'),
    ('དོགས་ གཅོད་', 'དོགས་གཅོད་'),
    ('དོན་ དམ་', 'དོན་དམ་'),
    ('དོན་བྱེད་པ འི་ གནས་སྐབས་ རྣམས་ དང་ལྡན་པ་', 'དོན་ བྱེད་པ འི་ གནས་སྐབས་ རྣམས་ དང་ ལྡན་པ་'),
    ('དོན་མེད་', 'དོན་ མེད་'),
    ('དྲི་ མ འི་ བདག་ཉིད་ ལས་ རིང་ བ', 'དྲི་མ འི་ བདག་ཉིད་ ལས་ རིང་བ'),
    ('དྲི་མེད་', 'དྲི་ མེད་'),
    ('དྲི་མ་མེད་པ', 'དྲི་མ་ མེད་པ'),
    ('དྲི་མ་མེད་པ་', 'དྲི་མ་ མེད་པ་'),
    ('དྲི་མ་མེད་པ་ ནི་ རྣམ་པ་ཐམས་ཅད་ རྣམ་པར་དག་ པ་ དང་ལྡན་པ', 'དྲི་མ་ མེད་པ་ ནི་ རྣམ་པ་ ཐམས་ཅད་ རྣམ་པ ར་ དག་པ་ དང་ ལྡན་པ'),
    ('དྲི་ཟའི་གྲོང་ཁྱེར་', 'དྲི་ཟ འི་ གྲོང་ཁྱེར་'),
    ('དྲིམ་ཅན', 'དྲིམ་ ཅན'),
    ('དྲུང་དུ་', 'དྲུང་ དུ་'),
    ('དྲྭ་ བ་', 'དྲྭ་བ་'),
    ('དྲྭ་ བས་', 'དྲྭ་བ ས་'),
    ('དྷྭ་ཡི་ བྱང་ཆུབ་ཤིང་', 'དྷྭ་ ཡི་ བྱང་ཆུབ་ ཤིང་'),
    ('དྷཱ་ཡི་ བྱང་ཆུབ་ཤིང་', 'དྷཱ་ ཡི་ བྱང་ཆུབ་ ཤིང་'),
    ('ན་ཚོད་', 'ན་ ཚོད་'),
    ('ནམ་ མཁའ', 'ནམ་མཁའ'),
    ('ནམ་ མཁའ་', 'ནམ་མཁའ་'),
    ('ནམ་ མཁའ་ ལྟར་ ཡོངས་ སུ་ ཆད་པ་ མེད་པ འི་ ངོ་བོ་ ཉིད་', 'ནམ་མཁའ་ ལྟར་ ཡོངས་ སུ་ ཆད་པ་ མེད་པ འི་ ངོ་བོ་ཉིད་'),
    ('ནམ་ཡང་', 'ནམ་ ཡང་'),
    ('ནམ་མཁའ་ལྡིང་', 'ནམ་མཁའ་ ལྡིང་'),
    ('ནའང༌', 'ན འང༌'),
    ('ནས་ནི', 'ནས་ ནི'),
    ('ནེ་གསིང་', 'ནེ་ གསིང་'),
    ('པདྨའི་', 'པདྨ འི་'),
    ('ཕ་ རོལ་', 'ཕ་རོལ་'),
    ('ཕ་དང་ མ་དང་', 'ཕ་ དང་ མ་ དང་'),
    ('ཕ་རོལ་ཚུ་རོལ་', 'ཕ་རོལ་ ཚུ་རོལ་'),
    ('ཕུལ་ བས', 'ཕུལ་བ ས'),
    ('ཕོངས་པར་', 'ཕོངས་པ ར་'),
    ('ཕྱག་འཚལ་བ', 'ཕྱག་ འཚལ་བ'),
    ('ཕྱག་འཚལ་བ འི་ བསྟོད་པ་ ཞེས་བྱ་བ', 'ཕྱག་ འཚལ་བ འི་ བསྟོད་པ་ ཞེས་ བྱ་བ'),
    ('ཕྱག་འཚལ', 'ཕྱག་ འཚལ'),
    ('ཕྱག་འཚལ་', 'ཕྱག་ འཚལ་'),
    ('ཕྱིན་གཞོལ་', 'ཕྱིན་ གཞོལ་'),
    ('ཕྱིན་པས', 'ཕྱིན་པ ས'),
    ('ཕྱིར་ཞིང་', 'ཕྱིར་ ཞིང་'),
    ('ཕྱོགས་ཀྱི་གླང་པོ', 'ཕྱོགས་ ཀྱི་ གླང་པོ'),
    ('ཕྱོགས་ཀྱི་གླང་པོ་', 'ཕྱོགས་ ཀྱི་ གླང་པོ་'),
    ('ཕྱོགས་གཅིག་', 'ཕྱོགས་ གཅིག་'),
    ('ཕྱོགས་བཅུ་', 'ཕྱོགས་ བཅུ་'),
    ('ཕྲ་ རབ་', 'ཕྲ་རབ་'),
    ('ཕྲ་ རྒྱས་', 'ཕྲ་རྒྱས་'),
    ('ཕྲད་པར་', 'ཕྲད་པ ར་'),
    ('བ་ གླང་རྭ་ ཡི་ དཔེས', 'བ་གླང་ རྭ་ ཡི་ དཔེ ས'),
    ('བ་ར་', 'བ་ ར་'),
    ('བཀག་ པ', 'བཀག་པ'),
    ('བཀའ་ བས་', 'བཀའ་བ ས་'),
    ('བཀའ་སྩལ་', 'བཀའ་ སྩལ་'),
    ('བག་མེད་', 'བག་ མེད་'),
    ('བགྱིད་པའི', 'བགྱིད་པ འི'),
    ('བགྱིས་པར', 'བགྱིས་པ ར'),
    ('བགྲོད་དཀའ་ བགྲོད་ མིན་ མི་རིག་པས', 'བགྲོད་ དཀའ་ བགྲོད་ མིན་ མི་ རིག་པ ས'),
    ('བགྲོད་པ་གཅིག་', 'བགྲོད་པ་ གཅིག་'),
    ('བཅས་པར', 'བཅས་པ ར'),
    ('བཅིངས་པས', 'བཅིངས་པ ས'),
    ('བཅོ་ ལྔ་', 'བཅོ་ལྔ་'),
    ('བཅོམ་ལྡན་', 'བཅོམ་ ལྡན་'),
    ('བཅོམ་ ལྡན་ འདས་ མ་', 'བཅོམ་ལྡན་འདས་མ་'),
    ('བཏང་ བ', 'བཏང་བ'),
    ('བཏང་ སྙོམས་', 'བཏང་སྙོམས་'),
    ('བཏུལ་ བ', 'བཏུལ་བ'),
    ('བདག་མེད་', 'བདག་ མེད་'),
    ('བདག་མེད་ སེང་གེའི་ང་རོ', 'བདག་ མེད་ སེང་གེ འི་ ང་རོ'),
    ('བདག་མེད་པ་ གང་ དབྱེར་མེད་པ', 'བདག་ མེད་པ་ གང་ དབྱེ ར་ མེད་པ'),
    ('བདག་ ཅག་', 'བདག་ཅག་'),
    ('བདག་ཉིད་ཆེན་པོ', 'བདག་ཉིད་ ཆེན་པོ'),
    ('བདག་ནི་', 'བདག་ ནི་'),
    ('བདག་ལ་', 'བདག་ ལ་'),
    ('བདུད་ལས་རྒྱལ་བ', 'བདུད་ ལས་ རྒྱལ་བ'),
    ('བདུད་རྩི་འཛག་ པ་', 'བདུད་རྩི་ འཛག་པ་'),
    ('བདེ་བ ར་ གཤེགས་ ཀྱི་ བདག་ཉིད་ཆེན་པོ་', 'བདེ་ བར་ གཤེགས་ ཀྱི་ བདག་ཉིད་ ཆེན་པོ་'),
    ('བདེ་ འགྲོ་', 'བདེ་འགྲོ་'),
    ('བདེ་བ་ཅན', 'བདེ་བ་ ཅན'),
    ('བདེ་བ་ཅན་', 'བདེ་བ་ ཅན་'),
    ('བདེན་དོན་', 'བདེན་ དོན་'),
    ('བཙལ་ བས་', 'བཙལ་བ ས་'),
    ('བཟུང་ བས', 'བཟུང་བ ས'),
    ('བར་ མ་', 'བར་མ་'),
    ('བར་མ་ཆད་པ', 'བར་ མ་ ཆད་པ'),
    ('བརྒྱ་ བྱིན་', 'བརྒྱ་བྱིན་'),
    ('བརྒྱད་ པོ་', 'བརྒྱད་པོ་'),
    ('བརྒྱན་པའི', 'བརྒྱན་པ འི'),
    ('བརྗོད་པའི་སྤྱོད་ཡུལ་', 'བརྗོད་པ འི་ སྤྱོད་ཡུལ་'),
    ('བརྗོད་པར་', 'བརྗོད་པ ར་'),
    ('བརྟག་ པ་', 'བརྟག་པ་'),
    ('བརྩེ་ བ', 'བརྩེ་བ'),
    ('བརྩོན་པས', 'བརྩོན་པ ས'),
    ('བསགས་པའོ', 'བསགས་པ འོ'),
    ('བསེ་རུའི་རྭ་', 'བསེ་རུ འི་ རྭ་'),
    ('བསོད་ཉམས་', 'བསོད་ ཉམས་'),
    ('བསོད་དང་', 'བསོད་ དང་'),
    ('བསོད་ནམ ས་', 'བསོད་ནམས་'),
    ('བསྐྱབ་པར་', 'བསྐྱབ་པ ར་'),
    ('བསྒྱུར་ བའོ', 'བསྒྱུར་བ འོ'),
    ('བསྟེན་པས', 'བསྟེན་པ ས'),
    ('བསྟོད་པའི', 'བསྟོད་པ འི'),
    ('བསྲུང་ བ་', 'བསྲུང་བ་'),
    ('བུ་དང་', 'བུ་ དང་'),
    ('བེམས་ པོ་', 'བེམས་པོ་'),
    ('བོད་ སྐད་', 'བོད་སྐད་'),
    ('བྱ་ བ་ དང་ བྱེད་པ འི་ མཚན་ཉིད་ དུ་ འབྲེལ་བ་ ནི་ འབྲེལ་ བའོ', 'བྱ་བ་ དང་ བྱེད་པ འི་ མཚན་ཉིད་ དུ་ འབྲེལ་བ་ ནི་ འབྲེལ་བ འོ'),
    ('བྱང་ ཆུབ་', 'བྱང་ཆུབ་'),
    ('བྱང་ ཆུབ་ སེམས་ ལ་ ཕྱག་ བྱའོ', 'བྱང་ཆུབ་ སེམས་ ལ་ ཕྱག་ བྱ འོ'),
    ('བྱང་ཆུབ་པ', 'བྱང་ ཆུབ་པ'),
    ('བྱང་ཆུབ་ཆེན་པོ', 'བྱང་ཆུབ་ ཆེན་པོ'),
    ('བྱང་ཆུབ་མཆོག་', 'བྱང་ཆུབ་ མཆོག་'),
    ('བྱང་ཆུབ་ སེམས་དཔའ', 'བྱང་ཆུབ་སེམས་དཔའ'),
    ('བྱང་ཆུབ་ སེམས་དཔའ འི་ ཚོགས་ ཏེ་ དེ་ རྣམས་ ཀྱི་ ནང་ དུའོ', 'བྱང་ཆུབ་སེམས་དཔའ འི་ ཚོགས་ ཏེ་ དེ་ རྣམས་ ཀྱི་ ནང་ དུ འོ'),
    ('བྱང་ཆུབ་ སེམས་དཔའ་', 'བྱང་ཆུབ་སེམས་དཔའ་'),
    ('བྱང་ཆུབ་སྙིང་པོ', 'བྱང་ཆུབ་ སྙིང་པོ'),
    ('བྱའོ', 'བྱ འོ'),
    ('བྱས་པ་གཟོ་བ', 'བྱས་པ་ གཟོ་བ'),
    ('བྱས་པ་ཅན་', 'བྱས་པ་ ཅན་'),
    ('བྱས་པས', 'བྱས་པ ས'),
    ('བྱུང་ བར', 'བྱུང་བ ར'),
    ('བྱེ་ བ་', 'བྱེ་བ་'),
    ('བྱེ་ བར་', 'བྱེ་བ ར་'),
    ('བྱེ་ བྲག་ ཇི་ ལྟ་བུ་', 'བྱེ་བྲག་ ཇི་ལྟ་བུ་'),
    ('བྱེད་པའི', 'བྱེད་པ འི'),
    ('བྱེད་པའོ', 'བྱེད་པ འོ'),
    ('བྲག་ ཅ་', 'བྲག་ཅ་'),
    ('བླ་ན་མེད་པ', 'བླ་ ན་ མེད་པ'),
    ('བླ་ན་མེད་པ་', 'བླ་ ན་ མེད་པ་'),
    ('བླ་མེད་', 'བླ་ མེད་'),
    ('བླངས་པར', 'བླངས་པ ར'),
    ('བློ་ལྡན་', 'བློ་ ལྡན་'),
    ('བློ་ ལྡན་ ཁྱོད་ ཀྱི་ ཕུང་ དེའང༌', 'བློ་ལྡན་ ཁྱོད་ ཀྱི་ ཕུང་ དེ འང༌'),
    ('མ་མཆིས་པ', 'མ་ མཆིས་པ'),
    ('མ་མཆིས་པ འང་ མ་ ལགས་སོ', 'མ་ མཆིས་པ འང་ མ་ ལགས་ སོ'),
    ('མ་འོངས་པ་', 'མ་ འོངས་པ་'),
    ('མ་ཡིན་པ', 'མ་ ཡིན་པ'),
    ('མ་ཡིན་པ་', 'མ་ ཡིན་པ་'),
    ('མ་ལགས', 'མ་ ལགས'),
    ('མཁྱེན་པས', 'མཁྱེན་པ ས'),
    ('མང་ བ་', 'མང་བ་'),
    ('མངའ་ བ', 'མངའ་བ'),
    ('མང ར་ པོ་', 'མངར་པོ་'),
    ('མངོན་ སུམ་', 'མངོན་སུམ་'),
    ('མངོན་པར་ མཐོ་བ འི་ ངོ་བོ་ འཁོར་ལོ ས་ སྒྱུར་བ་ ཉིད་ ལ་ སོགས་པ་', 'མངོན་པ ར་ མཐོ་བ འི་ ངོ་བོ་ འཁོར་ལོ ས་ སྒྱུར་བ་ ཉིད་ ལ་སོགས་པ་'),
    ('མཆོག་གི་དངོས་གྲུབ་', 'མཆོག་ གི་ དངོས་གྲུབ་'),
    ('མཆོག་གི་དངོས་གྲུབ་ གསོལ་བ་ གདབ་པ ར་ འོས་པ་ དེ་དག་ ཀྱང་ གང་ཞེ་', 'མཆོག་ གི་ དངོས་གྲུབ་ གསོལ་བ་ གདབ་པ ར་ འོས་པ་ དེ་དག་ ཀྱང་ གང་ ཞེ་'),
    ('མཆོག་ཏུ་གྱུར་པ་', 'མཆོག་ ཏུ་ གྱུར་པ་'),
    ('མཆོག་དགའ་ ལྡན་ གྱི་ གཞལ་ཡས་ཁང་ ན་ བཞུགས་པ་ བྱང་ཆུབ་སེམས་དཔའ་ དམ་པ་ཏོག་', 'མཆོག་ དགའ་ལྡན་ གྱི་ གཞལ་ཡས་ ཁང་ ན་ བཞུགས་པ་ བྱང་ཆུབ་ སེམས་དཔའ་ དམ་པ་ ཏོག་'),
    ('མཆོག་སྦྱིན་ མཛད་པ་ རྒྱལ་བ་མཆོག', 'མཆོག་ སྦྱིན་ མཛད་པ་ རྒྱལ་བ་ མཆོག'),
    ('མཆོད་རྟེན་རྣམ་དག་', 'མཆོད་རྟེན་ རྣམ་དག་'),
    ('མཉམ་མེད་', 'མཉམ་ མེད་'),
    ('མཐའ་ཅན', 'མཐའ་ ཅན'),
    ('མཐའ་ཅན་', 'མཐའ་ ཅན་'),
    ('མཐའ་ མ་', 'མཐའ་མ་'),
    ('མཐའ་མེད', 'མཐའ་ མེད'),
    ('མཐར', 'མཐ ར'),
    ('མཐར་', 'མཐ ར་'),
    ('མཐོང་ བ', 'མཐོང་བ'),
    ('མཐོང་ བ་', 'མཐོང་བ་'),
    ('མཐོང་ བ་ ཙམ་ གྱིས་ དད་ འགྱུར་ བར', 'མཐོང་བ་ ཙམ་ གྱིས་ དད་ འགྱུར་བ ར'),
    ('མདོ་ སྡེ', 'མདོ་སྡེ'),
    ('མཚང་འབྲུ་བ་', 'མཚང་ འབྲུ་བ་'),
    ('མཚན་གྱི་མེ་ཏོག་', 'མཚན་ གྱི་ མེ་ཏོག་'),
    ('མཚན་ མ་', 'མཚན་མ་'),
    ('མཚུངས་པར་', 'མཚུངས་པ ར་'),
    ('མཚོ་ཆེན་', 'མཚོ་ ཆེན་'),
    ('མཚོན་ བྱ་', 'མཚོན་བྱ་'),
    ('མཛད པ', 'མཛདཔ'),
    ('མཛད་པ་བཅུ་གཉིས་', 'མཛད་པ་ བཅུ་གཉིས་'),
    ('མཛད་པ་བཅུ་གཉིས་ ཀྱི་ ཚུལ་ ལ་ བསྟོད་པ་ ཞེས་བྱ་བ་', 'མཛད་པ་ བཅུ་གཉིས་ ཀྱི་ ཚུལ་ ལ་ བསྟོད་པ་ ཞེས་ བྱ་བ་'),
    ('མི་བསྐྱོད་པ', 'མི་ བསྐྱོད་པ'),
    ('མི་མཐུན་པ', 'མི་ མཐུན་པ'),
    ('མི་མཐུན་པ་', 'མི་ མཐུན་པ་'),
    ('མི་འགལ་བ་', 'མི་ འགལ་བ་'),
    ('མི་འདའ', 'མི་ འདའ'),
    ('མི་འདོད་པ་', 'མི་ འདོད་པ་'),
    ('མི་ རིགས', 'མི་རིགས'),
    ('མི་རིགས་པ་', 'མི་ རིགས་པ་'),
    ('མི་སྡུག་ པ་', 'མི་ སྡུག་པ་'),
    ('མི་སྤྱོད་', 'མི་ སྤྱོད་'),
    ('མི་སྤྱོད་ རྣམ་ མི་སྤྱོད', 'མི་ སྤྱོད་ རྣམ་ མི་ སྤྱོད'),
    ('མི་སྤྱོད་ པ་', 'མི་ སྤྱོད་པ་'),
    ('མི་སློབ་པ་', 'མི་ སློབ་པ་'),
    ('མི་གཡོ་', 'མི་ གཡོ་'),
    ('མི་འཇིགས་པ་', 'མི་ འཇིགས་པ་'),
    ('མིན་པས', 'མིན་པ ས'),
    ('མུ་ སྟེགས་', 'མུ་སྟེགས་'),
    ('མུ་ སྟེགས་ ཕམ་ མཛད་ མཆོད་རྟེན་ ཕྱག་འཚལ་', 'མུ་སྟེགས་ ཕམ་ མཛད་ མཆོད་རྟེན་ ཕྱག་ འཚལ་'),
    ('མེ་ཁྱེར་ ཙམ་ དུ་ མི་སྤྱོད་', 'མེ་ ཁྱེར་ ཙམ་ དུ་ མི་ སྤྱོད་'),
    ('མེ་ ཏོག་ གིས་ བརྒྱན་པ་ ལས་ བསོད་ནམས་ གང་ ཐོབ་པར', 'མེ་ཏོག་ གིས་ བརྒྱན་པ་ ལས་ བསོད་ནམས་ གང་ ཐོབ་པ ར'),
    ('མེད་པས', 'མེད་པ ས'),
    ('མོས་པས་སྤྱོད་པ་', 'མོས་པ ས་ སྤྱོད་པ་'),
    ('མྱ་ ངན་', 'མྱ་ངན་'),
    ('མྱ་ ངན་ ཐམས་ཅད་ ངེས་པ ར་ འཇོམས་པ འི་ བློ་གྲོས་ ལ་ སོགས་པའོ', 'མྱ་ངན་ ཐམས་ཅད་ ངེས་པ ར་ འཇོམས་པ འི་ བློ་གྲོས་ ལ་སོགས་པ འོ'),
    ('མྱ་ངན་མེད་', 'མྱ་ངན་ མེད་'),
    ('མྱུར་དུ་', 'མྱུར་ དུ་'),
    ('ཚད་ མ་', 'ཚད་མ་'),
    ('ཚད་མེད་', 'ཚད་ མེད་'),
    ('ཚད་མེད་ པ་', 'ཚད་ མེད་པ་'),
    ('ཚར་བཅད་', 'ཚར་ བཅད་'),
    ('ཚིགས་ བཅད་', 'ཚིགས་བཅད་'),
    ('ཚིམ་པས', 'ཚིམ་པ ས'),
    ('ཚོར་ བ་', 'ཚོར་བ་'),
    ('ཞལ་གྱིས་བཞེས', 'ཞལ་ གྱིས་ བཞེས'),
    ('ཞུ་ ཆེན་ གྱི་ ལོ་ཙཱ་བ་ དགེ་སློང་ བསོད་ནམས་ བཟང་པོ', 'ཞུ་ཆེན་ གྱི་ ལོ་ཙཱ་བ་ དགེ་སློང་ བསོད་ནམས་བཟང་པོ'),
    ('ཞུགས་པར་', 'ཞུགས་པ ར་'),
    ('ཞེས་བྱ་བ', 'ཞེས་ བྱ་བ'),
    ('ཞེས་བྱ་བ་', 'ཞེས་ བྱ་བ་'),
    ('ཞེས་བྱ་བ་ གླེང་ བསླང་བ འི་ ཚིགས་ སུ་ བཅད་པ་ འདི ས་ ནི་ བསྡུས་པ འི་ དོན་ བཞི་པོ་ འདི་ལྟ་སྟེ', 'ཞེས་ བྱ་བ་ གླེང་ བསླང་བ འི་ ཚིགས་ སུ་ བཅད་པ་ འདི ས་ ནི་ བསྡུས་པ འི་ དོན་ བཞི་པོ་ འདི་ ལྟ་ སྟེ'),
    ('ཞེས་བྱ་བ་ འདི་དག་ གིས་ ནི་ ཆོས་སྟོན་པ', 'ཞེས་ བྱ་བ་ འདི་དག་ གིས་ ནི་ ཆོས་ སྟོན་པ'),
    ('ཞེས་བྱ་བ་ འདི་དག་ གིས་ ནི་ རང་བཞིན་ ཡོངས་སུ་དག་ པ', 'ཞེས་ བྱ་བ་ འདི་དག་ གིས་ ནི་ རང་བཞིན་ ཡོངས་ སུ་ དག་པ'),
    ('ཞེས་ པ་', 'ཞེས་པ་'),
    ('ཟག་མེད་', 'ཟག་ མེད་'),
    ('ཟབ་ མོ་', 'ཟབ་མོ་'),
    ('ཟླ་ བ་', 'ཟླ་བ་'),
    ('ཟླ་བ་པདྨས་', 'ཟླ་བ་ པདྨ ས་'),
    ('འཁུ་ བ་', 'འཁུ་བ་'),
    ('འཁོར་ བ', 'འཁོར་བ'),
    ('འཁོར་ བ་', 'འཁོར་བ་'),
    ('འཁོར་ མོ་', 'འཁོར་མོ་'),
    ('འཁོར་འདབ་', 'འཁོར་ འདབ་'),
    ('འཁོར་ ལོ', 'འཁོར་ལོ'),
    ('འཁོར་བའི་རྒྱ་མཚོ་', 'འཁོར་བ འི་ རྒྱ་མཚོ་'),
    ('འཁོར་བ་ མ་', 'འཁོར་བ་མ་'),
    ('འགའ་ཡང་', 'འགའ་ ཡང་'),
    ('འགའ་ཡང་ བཀག་ པ་', 'འགའ་ ཡང་ བཀག་པ་'),
    ('འགལ་བའི་དཔེ་', 'འགལ་བ འི་ དཔེ་'),
    ('འགྱུར་ བ', 'འགྱུར་བ'),
    ('འགྲན་ ཟླ་མེད་པ', 'འགྲན་ཟླ་ མེད་པ'),
    ('འགྲུབ་པའོ', 'འགྲུབ་པ འོ'),
    ('འགྲོ་ བ་', 'འགྲོ་བ་'),
    ('འགྲོ་ བ་ ཕན་པ ར་ མཛད་ ཀྱང་ བདག་ སྡུག་པར', 'འགྲོ་བ་ ཕན་པ ར་ མཛད་ ཀྱང་ བདག་ སྡུག་པ ར'),
    ('འགྲོ་ལ་ཕན་ཕྱིར་', 'འགྲོ་ ལ་ ཕན་ ཕྱིར་'),
    ('འགྲོ་བའི་མགོན་པོ་', 'འགྲོ་བ འི་ མགོན་པོ་'),
    ('འགྲོ་བ་ལྔ་ པོ་', 'འགྲོ་བ་ ལྔ་པོ་'),
    ('འགྲོགས་ པ་', 'འགྲོགས་པ་'),
    ('འང་ཅིའི་ཕྱིར་', 'འང་ ཅི འི་ ཕྱིར་'),
    ('འང་རྣམ་པ་ཐམས་ཅད་', 'འང་ རྣམ་པ་ ཐམས་ཅད་'),
    ('འཆད་པའོ', 'འཆད་པ འོ'),
    ('འཇམ་ དཔལ་', 'འཇམ་དཔལ་'),
    ('འཇམ་ དཔལ་ གཞོན་ ནུར་', 'འཇམ་དཔལ་ གཞོན་ནུ ར་'),
    ('འཇམ་ དཔལ་ གཞོན་ ནུར་ གྱུར་པ་ ལ་ ཕྱག་འཚལ་', 'འཇམ་དཔལ་ གཞོན་ནུ ར་ གྱུར་པ་ ལ་ ཕྱག་ འཚལ་'),
    ('འཇམ་དཔལ་ དབྱངས་ ལ་ ཕྱག་འཚལ་', 'འཇམ་དཔལ་དབྱངས་ ལ་ ཕྱག་ འཚལ་'),
    ('འཇིག་ རྟེན་', 'འཇིག་རྟེན་'),
    ('འཇིག་ རྟེན་ ལས་ འདས་པ་ སྟེ་ འཇིག་རྟེན་ ལས་ རྒལ་ བའོ', 'འཇིག་རྟེན་ ལས་ འདས་པ་ སྟེ་ འཇིག་རྟེན་ ལས་ རྒལ་བ འོ'),
    ('འཇིག་ རྟེན་པ་', 'འཇིག་རྟེན་པ་'),
    ('འཇིག་པར་', 'འཇིག་པ ར་'),
    ('འཇིག་རྟེན་གསུམ་', 'འཇིག་རྟེན་ གསུམ་'),
    ('འཇིག་རྟེན་གསུམ་ པོ་', 'འཇིག་རྟེན་ གསུམ་པོ་'),
    ('འཇིག་རྟེན་མགོན་', 'འཇིག་རྟེན་ མགོན་'),
    ('འཇིག་རྟེན་མགོན་ པོ་', 'འཇིག་རྟེན་ མགོན་པོ་'),
    ('འཇུག་པའི', 'འཇུག་པ འི'),
    ('འཇོམས་པས', 'འཇོམས་པ ས'),
    ('འཐུང་བྱེད་', 'འཐུང་ བྱེད་'),
    ('འདབ་ མ་', 'འདབ་མ་'),
    ('འདའ་ བར་', 'འདའ་བ ར་'),
    ('འདི་ ཉིད་', 'འདི་ཉིད་'),
    ('འདི་ ཉིད་ དེ་ ཉིད་', 'འདི་ཉིད་ དེ་ཉིད་'),
    ('འདི་ དག', 'འདི་དག'),
    ('འདི་ དག་', 'འདི་དག་'),
    ('འདི་ནི་', 'འདི་ ནི་'),
    ('འདི་ལྟ་བུ་ ཡིས་ སྣོད་ དུ་ རབ་ གྱུར་ཅིག', 'འདི་ ལྟ་བུ་ ཡིས་ སྣོད་ དུ་ རབ་ གྱུར་ ཅིག'),
    ('འདི་ ལྟར་ སངས་རྒྱས་ བཅོམ་ལྡན་ འདས་', 'འདི་ལྟར་ སངས་རྒྱས་ བཅོམ་ལྡན་འདས་'),
    ('འདིར་', 'འདི ར་'),
    ('འདིར་ བསྡུས་པ འི་ དོན་ གྱི་ དེ་ ཁོ་ན་ཉིད་', 'འདི ར་ བསྡུས་པ འི་ དོན་ གྱི་ དེ་ཁོ་ན་ཉིད་'),
    ('འདིར་ ཚིག་ གི་ མདོ་ ནི་ དེ་ལྟ་ བུ འི་ བདེ་བ ར་ གཤེགས་པ་', 'འདི ར་ ཚིག་ གི་ མདོ་ ནི་ དེ་ལྟ་བུ འི་ བདེ་བར་གཤེགས་པ་'),
    ('འདིས', 'འདི ས'),
    ('འདིས་', 'འདི ས་'),
    ('འདིས་ ནི་ དེ་བཞིན་ གཤེགས་པ་ རྣམས་ ཀྱི་ དེ་བཞིན་ ཉིད་', 'འདི ས་ ནི་ དེ་བཞིན་གཤེགས་པ་ རྣམས་ ཀྱི་ དེ་བཞིན་ཉིད་'),
    ('འདུ་ འཛི་', 'འདུ་འཛི་'),
    ('འདུས་ བྱས་', 'འདུས་བྱས་'),
    ('འདུས་ མ་ བྱས་', 'འདུས་མ་བྱས་'),
    ('འདོད་ ཆགས་', 'འདོད་ཆགས་'),
    ('འདོད་ ཆགས་ ལ་ སོགས་པ', 'འདོད་ཆགས་ ལ་སོགས་པ'),
    ('འདོད་ཆགས་བྲལ་བ', 'འདོད་ཆགས་ བྲལ་བ'),
    ('འདོད་པས', 'འདོད་པ ས'),
    ('འདྲ་ བ', 'འདྲ་བ'),
    ('འདྲ་ བ་', 'འདྲ་བ་'),
    ('འདྲ་ བར་', 'འདྲ་བ ར་'),
    ('འཕགས་ མ་', 'འཕགས་མ་'),
    ('འཕགས་ལམ་', 'འཕགས་ ལམ་'),
    ('འཕྱེ་ བོ་ དང་ ལོང་བ་ ལ་ སོགས་པ་', 'འཕྱེ་བོ་ དང་ ལོང་བ་ ལ་སོགས་པ་'),
    ('འབའ་ ཞིག་', 'འབའ་ཞིག་'),
    ('འབར་ བ', 'འབར་བ'),
    ('འབིགས་ བྱེད་', 'འབིགས་བྱེད་'),
    ('འབྱུང་ བ་', 'འབྱུང་བ་'),
    ('འབྱུང་ བ་ མིག་ གིས་ གཟུང་ མིན་པས', 'འབྱུང་བ་ མིག་ གིས་ གཟུང་ མིན་པ ས'),
    ('འབྱེད་པར་', 'འབྱེད་པ ར་'),
    ('འབྲས་བུ་གཉིས་', 'འབྲས་བུ་ གཉིས་'),
    ('འབྲས་བུ་གཉིས་ དང་ མཐུན་པ འི་ ཡོན་ཏན་ དང་ལྡན་པ་', 'འབྲས་བུ་ གཉིས་ དང་ མཐུན་པ འི་ ཡོན་ཏན་ དང་ ལྡན་པ་'),
    ('འབྲས་བུ་གསུམ་', 'འབྲས་བུ་ གསུམ་'),
    ('འབྲས་བུ་འབྱུང་', 'འབྲས་བུ་ འབྱུང་'),
    ('འབྲེལ་ བའམ', 'འབྲེལ་བ འམ'),
    ('འབྲེལ་ པ', 'འབྲེལ་པ'),
    ('འབྲེལ་ པ་', 'འབྲེལ་པ་'),
    ('འཚལ་ བ', 'འཚལ་བ'),
    ('འཚལ་ བས', 'འཚལ་བ ས'),
    ('འཛམ་ བུ་ ཆུ་བོ འི་ གསེར་ ལྟར་ ལྷང་ངེ་ བ', 'འཛམ་བུ་ ཆུ་བོ འི་ གསེར་ ལྟར་ ལྷང་ངེ་བ'),
    ('འོ་ ན', 'འོ་ན'),
    ('འོད་ ཟེར་ ཕྲེང་བ་ སྟོང་ ལྡན་པའི', 'འོད་ཟེར་ ཕྲེང་བ་ སྟོང་ ལྡན་པ འི'),
    ('འོན་ཀྱང་', 'འོན་ ཀྱང་'),
    ('འོན་ཏེ་', 'འོན་ ཏེ་'),
    ('འོལ་ མི་', 'འོལ་མི་'),
    ('ཡང ས་', 'ཡངས་'),
    ('ཡང་ དག་ ཇི་བཞིན་ རིག་ གྱུར་པའོ', 'ཡང་དག་ ཇི་བཞིན་ རིག་ གྱུར་པ འོ'),
    ('ཡང་ དག་ དོན་ ལ་ ཞུགས་ གྱུར་པས', 'ཡང་དག་ དོན་ ལ་ ཞུགས་ གྱུར་པ ས'),
    ('ཡང་དག་རིག་ པར་', 'ཡང་དག་ རིག་པ ར་'),
    ('ཡང་དག་རིག་ མ་', 'ཡང་དག་ རིག་མ་'),
    ('ཡན་ལག་བརྒྱད་ པ', 'ཡན་ལག་ བརྒྱད་པ'),
    ('ཡི་ གེ་', 'ཡི་གེ་'),
    ('ཡིད་བཞིན་ནོར་བུ་', 'ཡིད་ བཞིན་ ནོར་བུ་'),
    ('ཡིན་ཡང༌', 'ཡིན་ ཡང༌'),
    ('ཡིན་པས', 'ཡིན་པ ས'),
    ('ཡུན་རིང་', 'ཡུན་ རིང་'),
    ('ཡུལ་དྲུག་ སྣང་བ་མེད་པ', 'ཡུལ་ དྲུག་ སྣང་བ་ མེད་པ'),
    ('ཡེ་ ཤེས་', 'ཡེ་ཤེས་'),
    ('ཡེ་ ཤེས་རྒྱ་ མཚོ་', 'ཡེ་ཤེས་ རྒྱ་མཚོ་'),
    ('ཡེ་ཤེས་ཀྱི་ཚོགས་', 'ཡེ་ཤེས་ ཀྱི་ ཚོགས་'),
    ('ཡོངས་སུ་དག་ གྱུར་པ ས', 'ཡོངས་ སུ་ དག་ གྱུར་པས'),
    ('ཡོངས་སུ་དག་ པ', 'ཡོངས་ སུ་ དག་པ'),
    ('ཡོངས་སུ་དག་ པ འི་ བདག་ཉིད་ ཅན་ བརྩེ་བ་ ཆེན་པོ འི་ དབང་ དུ་གྱུར་པ', 'ཡོངས་ སུ་ དག་པ འི་ བདག་ཉིད་ ཅན་ བརྩེ་བ་ ཆེན་པོ འི་ དབང་ དུ་ གྱུར་པ'),
    ('ཡོད་པའོ', 'ཡོད་པ འོ'),
    ('ཡོན་ ཏན་', 'ཡོན་ཏན་'),
    ('ཡོན་ ཏན་ ཇི་ ལྟ་བུ་', 'ཡོན་ཏན་ ཇི་ལྟ་བུ་'),
    ('ཡོན་ཏན་ཅན་', 'ཡོན་ཏན་ ཅན་'),
    ('ཡོན་ཏན་འོད་ ཟེར་', 'ཡོན་ཏན་ འོད་ཟེར་'),
    ('ཡོན་ཏན་རྒྱ་མཚོ་', 'ཡོན་ཏན་ རྒྱ་མཚོ་'),
    ('ར་ བ་', 'ར་བ་'),
    ('ར་གནས་', 'ར་ གནས་'),
    ('རང་གི་ངོ་བོ་', 'རང་ གི་ ངོ་བོ་'),
    ('རང་དོན་', 'རང་ དོན་'),
    ('རང་ བྱུང་ ཁྱོད་ ལ་ ཕྱག་འཚལ་', 'རང་བྱུང་ ཁྱོད་ ལ་ ཕྱག་ འཚལ་'),
    ('རང་རིག་', 'རང་ རིག་'),
    ('རང་རྒྱལ་ བ་', 'རང་ རྒྱལ་བ་'),
    ('རང་ སངས་རྒྱས་', 'རང་སངས་རྒྱས་'),
    ('རང་སེམས་', 'རང་ སེམས་'),
    ('རང་བཞིན་པས', 'རང་བཞིན་པ ས'),
    ('རངས་པའང༌', 'རངས་པ འང༌'),
    ('རད་རོད་', 'རད་ རོད་'),
    ('རབ་ཏུ་བྱུང་བ', 'རབ་ ཏུ་ བྱུང་བ'),
    ('རབ་དང་ ཡོངས་སུ་དག་ པ་', 'རབ་ དང་ ཡོངས་ སུ་ དག་པ་'),
    ('རབ་ཏུ་', 'རབ་ ཏུ་'),
    ('རབ་རིབ་འཇོམས་ པ', 'རབ་རིབ་ འཇོམས་པ'),
    ('རི་ དགས་', 'རི་དགས་'),
    ('རིག་ སྔགས་འཆང་', 'རིག་སྔགས་ འཆང་'),
    ('རིང་མིན་', 'རིང་ མིན་'),
    ('རིན་ ཆེན་', 'རིན་ཆེན་'),
    ('རིན་ཆེན་སྙིང་པོ་', 'རིན་ཆེན་ སྙིང་པོ་'),
    ('རིན་པོ་ ཆེ', 'རིན་པོ་ཆེ'),
    ('རིབ་དག', 'རིབ་ དག'),
    ('རིམ་གྱིས་', 'རིམ་ གྱིས་'),
    ('རེ་དོགས་', 'རེ་ དོགས་'),
    ('རེས་ འགའ་', 'རེས་འགའ་'),
    ('རོལ་མོ་མཁན་', 'རོལ་མོ་ མཁན་'),
    ('རྒལ་ བས', 'རྒལ་བ ས'),
    ('རྒོལ་ བ', 'རྒོལ་བ'),
    ('རྒོལ་ བ་', 'རྒོལ་བ་'),
    ('རྒྱ་ གར་', 'རྒྱ་གར་'),
    ('རྒྱ་ གར་སྐད་', 'རྒྱ་གར་ སྐད་'),
    ('རྒྱ་ཆེ་བ', 'རྒྱ་ ཆེ་བ'),
    ('རྒྱལ་ བར་ མཛད་པ་ ཅི་ ཞིག་', 'རྒྱལ་བ ར་ མཛད་པ་ ཅི་ཞིག་'),
    ('རྒྱལ་ བས་', 'རྒྱལ་བ ས་'),
    ('རྒྱལ་མཆོག་', 'རྒྱལ་ མཆོག་'),
    ('རྒྱལ་པོའི་ཁབ་ ཀྱི་ འོད་མའི་ཚལ', 'རྒྱལ་པོ འི་ ཁབ་ ཀྱི་ འོད་མ འི་ ཚལ'),
    ('རྒྱས་པདྨ་', 'རྒྱས་ པདྨ་'),
    ('རྒྱུ་ སྐ ར་', 'རྒྱུ་སྐར་'),
    ('རྗེ་ བཙུན་', 'རྗེ་བཙུན་'),
    ('རྗེ་བཙུན་འཇམ་དཔལ་', 'རྗེ་བཙུན་ འཇམ་དཔལ་'),
    ('རྗེ་བཙུན་འཇམ་དཔལ་ གྱི་ དོན་དམ་པ འི་ བསྟོད་པ་ ཞེས་བྱ་བ', 'རྗེ་བཙུན་ འཇམ་དཔལ་ གྱི་ དོན་དམ་པ འི་ བསྟོད་པ་ ཞེས་ བྱ་བ'),
    ('རྗེ་བཙུན་འཇམ་དཔལ་ གྱི་ དོན་དམ་པ འི་ བསྟོད་པ་ ཞེས་བྱ་བ་', 'རྗེ་བཙུན་ འཇམ་དཔལ་ གྱི་ དོན་དམ་པ འི་ བསྟོད་པ་ ཞེས་ བྱ་བ་'),
    ('རྗེའི་', 'རྗེ འི་'),
    ('རྗེས་ མཐུན་པ', 'རྗེས་མཐུན་པ'),
    ('རྗེས་སུ་དཔག་ པ', 'རྗེས་ སུ་ དཔག་པ'),
    ('རྗེས་སུ་མཐུན་པ', 'རྗེས་ སུ་ མཐུན་པ'),
    ('རྗེས་སུ་འབྲང་བ་', 'རྗེས་ སུ་ འབྲང་བ་'),
    ('རྗེས་ འབྲང་', 'རྗེས་འབྲང་'),
    ('རྟག་ཏུ་', 'རྟག་ ཏུ་'),
    ('རྟག་པ་ཉིད་', 'རྟག་པ་ ཉིད་'),
    ('རྟོག་ གེ་', 'རྟོག་གེ་'),
    ('རྡུལ་མེད་', 'རྡུལ་ མེད་'),
    ('རྣམ་གྲོལ་ ཡངས་པ་ཅན་', 'རྣམ་ གྲོལ་ ཡངས་པ་ ཅན་'),
    ('རྣམ་ བྱང་', 'རྣམ་བྱང་'),
    ('རྣམ་ རྟོག་ གཡོ་ བ་', 'རྣམ་རྟོག་ གཡོ་བ་'),
    ('རྣམ་ ཤེས་བྱ་ བ', 'རྣམ་ཤེས་ བྱ་བ'),
    ('རྣམ་གྲོལ་ཡེ་ཤེས་', 'རྣམ་གྲོལ་ ཡེ་ཤེས་'),
    ('རྣམ་པར་གྲོལ་བ', 'རྣམ་པ ར་ གྲོལ་བ'),
    ('རྣམ་པར་དག་ པ', 'རྣམ་པ ར་ དག་པ'),
    ('རྣམ་པར་དག་ པར་', 'རྣམ་པ ར་ དག་པ ར་'),
    ('རྣམ་པར་དག་ པ་', 'རྣམ་པ ར་ དག་པ་'),
    ('རྣམ་པར་དབྱེ་བ་', 'རྣམ་པ ར་ དབྱེ་བ་'),
    ('རྣམ་པར་འཚེ་བ', 'རྣམ་པ ར་ འཚེ་བ'),
    ('རྣམ་པར་རྟོག་ པ', 'རྣམ་པ ར་ རྟོག་པ'),
    ('རྣམ་པར་རྟོག་ པ་', 'རྣམ་པ ར་ རྟོག་པ་'),
    ('རྣམ་པར་རྟོག་ པ་ དང་བཅས་པ འི་ སྙིང་རྗེ་ དེ་ཉིད་ བསྒོམ་པ་ མཆོག་ གིས་ གོམས་པ་ དང་ལྡན་པ ས་ འབྲས་བུ་ སེམས་ཅན་ གྱི་ རྣམ་པར་རྟོག་ པ་', 'རྣམ་པ ར་ རྟོག་པ་ དང་ བཅས་པ འི་ སྙིང་རྗེ་ དེ་ཉིད་ བསྒོམ་པ་ མཆོག་ གིས་ གོམས་པ་ དང་ ལྡན་པ ས་ འབྲས་བུ་ སེམས་ཅན་ གྱི་ རྣམ་པ ར་ རྟོག་པ་'),
    ('རྣམ་པར་ཤེས་པ', 'རྣམ་པ ར་ ཤེས་པ'),
    ('རྣམ་པ་ཀུན་', 'རྣམ་པ་ ཀུན་'),
    ('རྣམ་པ་སྣ་ཚོགས་', 'རྣམ་པ་ སྣ་ཚོགས་'),
    ('རྣམ་པར་', 'རྣམ་པ ར་'),
    ('རྣམས་ལ', 'རྣམས་ ལ'),
    ('རྣལ་དུ་', 'རྣལ་ དུ་'),
    ('རྣལ་ འབྱོར་', 'རྣལ་འབྱོར་'),
    ('རྣལ་ འབྱོར་པ་', 'རྣལ་འབྱོར་པ་'),
    ('རྣལ་ འབྱོར་པ་ དེ་ མཐའ ར་ སོན་པས', 'རྣལ་འབྱོར་པ་ དེ་ མཐའ ར་ སོན་པ ས'),
    ('རྨད་བྱུང་', 'རྨད་ བྱུང་'),
    ('རྨི་ ལམ་', 'རྨི་ལམ་'),
    ('རྨི་ ལམ་ ལ་ སོགས་པ', 'རྨི་ལམ་ ལ་སོགས་པ'),
    ('རྨོངས་པས', 'རྨོངས་པ ས'),
    ('རྩི་ བས', 'རྩི་བ ས'),
    ('རྩུབ་མིན་', 'རྩུབ་ མིན་'),
    ('རྩྭ་ མཆོག་ གྲོང་', 'རྩྭ་མཆོག་གྲོང་'),
    ('རྫི་ བ', 'རྫི་བ'),
    ('རྫོགས་པའི་བྱང་ཆུབ་', 'རྫོགས་པ འི་ བྱང་ཆུབ་'),
    ('རྫོགས་པའི་སངས་རྒྱས་ ཆོས་ ཀྱི་ ཆུ་སྲིན་ དུར་ པ', 'རྫོགས་པ འི་ སངས་རྒྱས་ ཆོས་ ཀྱི་ ཆུ་སྲིན་ དུར་པ'),
    ('རྫོགས་པར་', 'རྫོགས་པ ར་'),
    ('ལ་ཆེན་ པོ་ ཡང་ ཡིན་པ ས་ ན་ ཆོས་ ཀྱི་ རྒྱལ་སྲིད་ ཆེན་པོ་ སྟེ་ ཁམས་གསུམ་པ་', 'ལ་ ཆེན་པོ་ ཡང་ ཡིན་པ ས་ ན་ ཆོས་ ཀྱི་ རྒྱལ་སྲིད་ ཆེན་པོ་ སྟེ་ ཁམས་ གསུམ་པ་'),
    ('ལ་ ལ', 'ལ་ལ'),
    ('ལ་ ལ་ལ ས་', 'ལ་ལ་ ལས་'),
    ('ལ་ སོགས་པའོ', 'ལ་སོགས་པ འོ'),
    ('ལ་སྨན', 'ལ་ སྨན'),
    ('ལ་སོ་ སོར་རྟོག་པ་', 'ལ་ སོ་སོ ར་ རྟོག་པ་'),
    ('ལགས་སོ', 'ལགས་ སོ'),
    ('ལགས་ པ', 'ལགས་པ'),
    ('ལན་ཅིག་', 'ལན་ ཅིག་'),
    ('ལམ་མཁྱེན་པ', 'ལམ་ མཁྱེན་པ'),
    ('ལམ་འབྲས་ བུ་', 'ལམ་ འབྲས་བུ་'),
    ('ལའང༌', 'ལ འང༌'),
    ('ལའམ', 'ལ འམ'),
    ('ལས་ཀྱི་སྒྲིབ་པ', 'ལས་ ཀྱི་ སྒྲིབ་པ'),
    ('ལས་དམ་ ཚིག་', 'ལས་ དམ་ཚིག་'),
    ('ལུང་མ་བསྟན་ པ་', 'ལུང་ མ་ བསྟན་པ་'),
    ('ལུས་ཅན་', 'ལུས་ ཅན་'),
    ('ལེགས་གསུང་', 'ལེགས་ གསུང་'),
    ('ལེགས་པར་', 'ལེགས་པ ར་'),
    ('ལོ་ ཙཱ་ བ་ དགེ་སློང་ ཚུལ་ཁྲིམས་ རྒྱལ་བ ས་ བསྒྱུར་ བའོ', 'ལོ་ཙཱ་བ་ དགེ་སློང་ ཚུལ་ཁྲིམས་ རྒྱལ་བ ས་ བསྒྱུར་བ འོ'),
    ('ལོ་ ཙཱ་ བ་ ཚུལ་ཁྲིམས་ རྒྱལ་བ ས་ བསྒྱུར་ ཅིང་ ཞུས་ ཏེ་ གཏན་ ལ་ ཕབ་པའོ', 'ལོ་ཙཱ་བ་ ཚུལ་ཁྲིམས་ རྒྱལ་བ ས་ བསྒྱུར་ ཅིང་ ཞུས་ ཏེ་ གཏན་ ལ་ ཕབ་པ འོ'),
    ('ལོག་པར་', 'ལོག་པ ར་'),
    ('ལོངས་ སྤྱོད་', 'ལོངས་སྤྱོད་'),
    ('ལོན་པའི', 'ལོན་པ འི'),
    ('ལྕི་དང་', 'ལྕི་ དང་'),
    ('ལྕི་ བ་', 'ལྕི་བ་'),
    ('ལྟ་ བ', 'ལྟ་བ'),
    ('ལྟ་ བ་', 'ལྟ་བ་'),
    ('ལྟ་ བ་ ཐམས་ཅད་ འགོག་ བྱེད་པའི', 'ལྟ་བ་ ཐམས་ཅད་ འགོག་ བྱེད་པ འི'),
    ('ལྟ་ བས', 'ལྟ་བ ས'),
    ('ལྡན་པའི', 'ལྡན་པ འི'),
    ('ལྡོག་པའོ', 'ལྡོག་པ འོ'),
    ('ལྡོན་ཁོ་ན', 'ལྡོན་ ཁོ་ན'),
    ('ལྷ་ མ་ ཡིན་ གྱིས་ གཏིང་ རྙེད་པའི', 'ལྷ་མ་ཡིན་ གྱིས་ གཏིང་ རྙེད་པ འི'),
    ('ལྷ་མི འི་ སྟོན་པ་ སྨན་པའི་རྒྱལ་པོ་', 'ལྷ་ མི འི་ སྟོན་པ་ སྨན་པ འི་ རྒྱལ་པོ་'),
    ('ལྷ་མི་', 'ལྷ་ མི་'),
    ('ལྷ་མིན་ སྤྱོད་པ་ ཐ་ དད་', 'ལྷ་ མིན་ སྤྱོད་པ་ ཐ་དད་'),
    ('ལྷག་བཅས་', 'ལྷག་ བཅས་'),
    ('ལྷག་མེད་', 'ལྷག་ མེད་'),
    ('ལྷག་པར་', 'ལྷག་པ ར་'),
    ('ཤིན་ཏུ་', 'ཤིན་ ཏུ་'),
    ('ཤིན་ཏུ་ རྣམ་པར་དག་ པ་', 'ཤིན་ ཏུ་ རྣམ་པ ར་ དག་པ་'),
    ('ཤེས་ བྱ་', 'ཤེས་བྱ་'),
    ('ཤེས་ རབ་', 'ཤེས་རབ་'),
    ('ཤེས་ལྡན་', 'ཤེས་ ལྡན་'),
    ('ས་གསུམ་ གྱི་ བླ་མ་ རྣམས་ ཐུགས་རྗེའི་བདག་ ཉིད་', 'ས་ གསུམ་ གྱི་ བླ་མ་ རྣམས་ ཐུགས་རྗེ འི་ བདག་ཉིད་'),
    ('ས་ བོན་ གྱུར་པ་ ཁམས་ དེ་ ཉིད', 'ས་བོན་ གྱུར་པ་ ཁམས་ དེ་ཉིད'),
    ('ས་འགྲོ་', 'ས་ འགྲོ་'),
    ('སངས་ རྒྱས་', 'སངས་རྒྱས་'),
    ('སངས་ རྒྱས་ ཕྱག་ གི་ ཉི་མ་ དེར', 'སངས་རྒྱས་ ཕྱག་ གི་ ཉི་མ་ དེ ར'),
    ('སུམ་ ཅུ་ རྩ་ གསུམ་', 'སུམ་ཅུ་རྩ་གསུམ་'),
    ('སུམ་ ཅུ་', 'སུམ་ཅུ་'),
    ('སེམས་ ཅན་', 'སེམས་ཅན་'),
    ('སེམས་ ཅན་ ངན་ ལ འང་ མཆོག་ གཞོལ་ བར', 'སེམས་ཅན་ ངན་ ལ འང་ མཆོག་ གཞོལ་བ ར'),
    ('སེམས་ ཅན་ མོས་པ་ སྣ་ཚོགས་པ་', 'སེམས་ཅན་ མོས་པ་ སྣ་ཚོགས་ པ་'),
    ('སེམས་ ཅན་ རྣམས་ ཀྱི་ སོ་ སོ་ རང་ གིས་ རིག་པ ར་ གྱུར་པ་ གཞོན་ ནུ་མ', 'སེམས་ཅན་ རྣམས་ ཀྱི་ སོ་སོ་ རང་ གིས་ རིག་པ ར་ གྱུར་པ་ གཞོན་ནུ་མ'),
    ('སེམས་ ཅན་ རྣམས་ ནི་ སྨིན་པ ར་ མཛད་ ཕྱིར་ ལ་ ལ་', 'སེམས་ཅན་ རྣམས་ ནི་ སྨིན་པ ར་ མཛད་ ཕྱིར་ ལ་ལ་'),
    ('སེམས་ ཅན་ རྣམས་ ནི་ སྲོགས་ (སྲོག་)ཆགས་', 'སེམས་ཅན་ རྣམས་ ནི་ སྲོགས་(སྲོག་ )ཆགས་'),
    ('སེམས་ཉིད་', 'སེམས་ ཉིད་'),
    ('སོ་ བ་', 'སོ་བ་'),
    ('སོ་ སོ', 'སོ་སོ'),
    ('སོ་ སོ ར་ རིག་ ན་ དེ་ ཉིད་', 'སོ་སོ ར་ རིག་ ན་ དེ་ཉིད་'),
    ('སོ་ སོ་', 'སོ་སོ་'),
    ('སོ་སོར་ངེས་པ་', 'སོ་སོ ར་ ངེས་པ་'),
    ('སོར་མོའི་ཕྲེང་བ་ འང་ཞི་', 'སོར་མོ འི་ ཕྲེང་བ་ འང་ ཞི་'),
    ('སྐད་ ཅིག་', 'སྐད་ཅིག་'),
    ('སྐལ་ཅན་', 'སྐལ་ ཅན་'),
    ('སྐལ་ཉེས་པ་', 'སྐལ་ ཉེས་པ་'),
    ('སྐུ་ གདུང་', 'སྐུ་གདུང་'),
    ('སྐུ་གསུམ་', 'སྐུ་ གསུམ་'),
    ('སྐུ་གསུམ་ ལ་ བསྟོད་པ་ ཞེས་བྱ་བ', 'སྐུ་ གསུམ་ ལ་ བསྟོད་པ་ ཞེས་ བྱ་བ'),
    ('སྐུ་གསུམ་ ལ་ བསྟོད་པ་ ཞེས་བྱ་བ་', 'སྐུ་ གསུམ་ ལ་ བསྟོད་པ་ ཞེས་ བྱ་བ་'),
    ('སྐུ་ མདོག་', 'སྐུ་མདོག་'),
    ('སྐུ་ ཚེ་', 'སྐུ་ཚེ་'),
    ('སྐུ་ སྲོག་', 'སྐུ་སྲོག་'),
    ('སྐུར་ པ་', 'སྐུར་པ་'),
    ('སྐུལ་འདེབས', 'སྐུལ་ འདེབས'),
    ('སྐྱབས་སུ་མཆི་བ', 'སྐྱབས་ སུ་ མཆི་བ'),
    ('སྐྱེ་ བ་', 'སྐྱེ་བ་'),
    ('སྐྱེ་བ་གཅིག་', 'སྐྱེ་བ་ གཅིག་'),
    ('སྐྱེ་ མ་', 'སྐྱེ་མ་'),
    ('སྐྱོ་ བ་', 'སྐྱོ་བ་'),
    ('སྐྱོན་མེད་', 'སྐྱོན་ མེད་'),
    ('སྐྱོན་ཡོན་', 'སྐྱོན་ ཡོན་'),
    ('སྒྱུ་ མ', 'སྒྱུ་མ'),
    ('སྒྱུ་ མ་', 'སྒྱུ་མ་'),
    ('སྒྱུ་ མ་ བཞིན་ དུ་ རྐྱེན་ འབྱུང་ བར', 'སྒྱུ་མ་ བཞིན་ དུ་ རྐྱེན་ འབྱུང་བ ར'),
    ('སྒྲ་ དབྱངས་ རྒྱལ་པོ་ དེ་ ལ་ ཕྱག་ འཚལ་ལོ', 'སྒྲ་དབྱངས་ རྒྱལ་པོ་ དེ་ ལ་ ཕྱག་ འཚལ་ ལོ'),
    ('སྒྲ་གཅན་གདོང་', 'སྒྲ་གཅན་ གདོང་'),
    ('སྒྲིབ་པ་ལྔ་', 'སྒྲིབ་པ་ ལྔ་'),
    ('སྔ་ནས་', 'སྔ་ ནས་'),
    ('སྔ་ཕྱི ར་', 'སྔ་ ཕྱིར་'),
    ('སྔ་ མ', 'སྔ་མ'),
    ('སྔོན་དུས་', 'སྔོན་ དུས་'),
    ('སྔོན་ཚེ་', 'སྔོན་ ཚེ་'),
    ('སྙིང་དུ་སྡུག', 'སྙིང་ དུ་ སྡུག'),
    ('སྙིང་ རྗེ', 'སྙིང་རྗེ'),
    ('སྙིང་ རྗེས་', 'སྙིང་རྗེ ས་'),
    ('སྙིང་པོ་མེད་པ', 'སྙིང་པོ་ མེད་པ'),
    ('སྙིང་རྗེ་མེད་པ་', 'སྙིང་རྗེ་ མེད་པ་'),
    ('སྟོང་ ཉིད་', 'སྟོང་ཉིད་'),
    ('སྟོང་པ་ ཉིད', 'སྟོང་པ་ཉིད'),
    ('སྟོང་པ་ ཉིད་', 'སྟོང་པ་ཉིད་'),
    ('སྟོང་པ་ ཉིད་ དང་ ཆེས་ ཟབ་པའི', 'སྟོང་པ་ཉིད་ དང་ ཆེས་ ཟབ་པ འི'),
    ('སྟོབས་བཅུ', 'སྟོབས་ བཅུ'),
    ('སྡང་ བས་', 'སྡང་བ ས་'),
    ('སྡུག་ བསྔལ་', 'སྡུག་བསྔལ་'),
    ('སྣ་ ཚོགས་', 'སྣ་ཚོགས་'),
    ('སྣ་ཚོགས་ པ', 'སྣ་ཚོགས་པ'),
    ('སྣང་བ་མེད་པ', 'སྣང་བ་ མེད་པ'),
    ('སྣོད་མིན་', 'སྣོད་ མིན་'),
    ('སྤངས་པའི', 'སྤངས་པ འི'),
    ('སྤངས་པར', 'སྤངས་པ ར'),
    ('སྤོ་ བ་', 'སྤོ་བ་'),
    ('སྤོབས་སྐྱེས་པ', 'སྤོབས་ སྐྱེས་པ'),
    ('སྤྱོད་ པོ', 'སྤྱོད་པོ'),
    ('སྤྲུལ་པའི་སྐུ', 'སྤྲུལ་པ འི་ སྐུ'),
    ('སྤྲོས་མེད་', 'སྤྲོས་ མེད་'),
    ('སྦྱོང་བྱེད་ པ', 'སྦྱོང་ བྱེད་པ'),
    ('སྨན་མཆོག་', 'སྨན་ མཆོག་'),
    ('སྨན་པའི་རྒྱལ་པོ་', 'སྨན་པ འི་ རྒྱལ་པོ་'),
    ('སྨིན་པར་ མཛད་པ འི་ ཕྱིར་ ཏེ་ ཡོངས་ སུ་ སྨིན་པ འི་ དོན་ དུའོ', 'སྨིན་པ ར་ མཛད་པ འི་ ཕྱིར་ ཏེ་ ཡོངས་ སུ་ སྨིན་པ འི་ དོན་ དུ འོ'),
    ('སྨོན་པའོ', 'སྨོན་པ འོ'),
    ('སྨོན་པར་', 'སྨོན་པ ར་'),
    ('སྨོས་ཅི', 'སྨོས་ ཅི'),
    ('སྨྲ་ བ', 'སྨྲ་བ'),
    ('སྨྲའོ', 'སྨྲ འོ'),
    ('སྩལ་ བའི', 'སྩལ་བ འི'),
    ('སྩལ་ བས', 'སྩལ་བ ས'),
    ('སྩོལ་ བའོ', 'སྩོལ་བ འོ'),
    ('སྲ ས་', 'སྲས་'),
    ('སྲས་ཀྱི་ཐུ་བོ ར་ དབང་བསྐུར་', 'སྲས་ ཀྱི་ ཐུ་བོ ར་ དབང་ བསྐུར་'),
    ('སྲི་ བ་', 'སྲི་བ་'),
    ('སྲིད་པ་གསུམ་', 'སྲིད་པ་ གསུམ་'),
    ('སྲིད་པ་གསུམ་ གྱི་ འཇིགས་ སེལ་ ཞེས་བྱ་བ་', 'སྲིད་པ་ གསུམ་ གྱི་ འཇིགས་ སེལ་ ཞེས་ བྱ་བ་'),
    ('སྲོག་གནས་ པ', 'སྲོག་ གནས་པ'),
    ('སླར་ཡང་', 'སླར་ ཡང་'),
    ('སླུ་ བ', 'སླུ་བ'),
    ('སློང་ བ', 'སློང་བ'),
    ('སློབ་ དཔོན་', 'སློབ་དཔོན་'),
    ('ཨེ་ མ་ ཡ་མཚན་ ཆེར་', 'ཨེ་མ་ ཡ་མཚན་ ཆེ ར་'),
    ('མ་རིག་ པ་', 'མ་ རིག་པ་'),
    ('ཀུ་ ཤ འི', 'ཀུ་ཤ འི'),
    ('ཀུ་ཤའི', 'ཀུ་ཤ འི'),
    ('ཀུན་ མཁྱེན', 'ཀུན་མཁྱེན'),
    ('ཕྱག་འཚལ', 'ཕྱག་ འཚལ'),
    ('ཀྱི་ ཧུད', 'ཀྱི་ཧུད'),
    ('གཟུགས་མཆོག', 'གཟུགས་ མཆོག'),
    ('ཁ་ ཅིག', 'ཁ་ཅིག'),
    ('ཕ་རོལ་འདོད', 'ཕ་རོལ་ འདོད'),
    ('བྱང་ཆུབ་གསུམ', 'བྱང་ཆུབ་ གསུམ'),
    ('ཅི་ རིགས', 'ཅི་རིགས'),
    ('ཞེས་བྱ་བ', 'ཞེས་ བྱ་བ'),
    ('ཁོ་བོས', 'ཁོ་བོ ས'),
    ('མཚམས་མེད་ ལྔ', 'མཚམས་ མེད་ ལྔ'),
    ('མཚམས་མེད་ལྔ', 'མཚམས་ མེད་ ལྔ'),
    ('ཁྱད་པ ར', 'ཁྱད་པར'),
    ('སོ་ སོ་ རང་རིག', 'སོ་སོ་ རང་ རིག'),
    ('སོ་སོ་རང་རིག', 'སོ་སོ་ རང་ རིག'),
    ('ཡོངས་ སུ་ དག་པ', 'ཡོངས་ སུ་ དག་པ'),
    ('ཡོངས་སུ་དག་ པ', 'ཡོངས་ སུ་ དག་པ'),
    ('དུ་ གྱུར་པ', 'དུ་ གྱུར་པ'),
    ('དུ་གྱུར་པ', 'དུ་ གྱུར་པ'),
    ('ཕུན་སུམ་ཚོགས་པ', 'ཕུན་སུམ་ ཚོགས་པ'),
    ('སོ་སོ་ཡང་དག་ པར', 'སོ་སོ་ ཡང་དག་པ ར'),
    ('བྱའོ', 'བྱ འོ'),
    ('ཐུགས་རྗེའི་བདག་ ཉིད་ ', 'ཐུགས་རྗེ འི་ བདག་ཉིད'),
    ('ངོ་བོ་ཉིད', 'ངོ་བོ་ ཉིད'),
    ('ཇི་ལྟ་བ་བཞིན', 'ཇི་ལྟ་བ་ བཞིན'),
    ('རྣམ་པར་མི་རྟོག', 'རྣམ་པ ར་ མི་ རྟོག'),
    ('བདག་ཉིད་ཆེ་ བ', 'བདག་ཉིད་ ཆེ་བ'),
    ('མངོན་འདོད་པས', 'མངོན་ འདོད་པ ས'),
    ('མངོན་འདོད་ པ ས', 'མངོན་ འདོད་པ ས'),
    ('མི་མཐུན་པ', 'མི་ མཐུན་པ'),
    ('འཇོམས་པའི', 'འཇོམས་པ འི'),
    ('ཞི་ བའམ', 'ཞི་བ འམ'),
    ('གཉིད་ལོག་པས', 'གཉིད་ ལོག་པ ས'),
    ('གཉིས་ ཀ་ལ ས', 'གཉིས་ཀ་ ལས'),
    ('ལས་མི', 'ལས་ མི'),
    ('གདུལ་ བྱ', 'གདུལ་བྱ'),
    ('གཙང་ མ', 'གཙང་མ'),
    ('ཐབས་མེད་པ', 'ཐབས་ མེད་པ'),
    ('བཅུར', 'བཅུ ར'),
    ('ཚོགས་ཆེན', 'ཚོགས་ ཆེན'),
    ('ལམ་བཟང་པོ', 'ལམ་ བཟང་པོ')
]


In [5]:
propernouns = [
    (r'ཀུ་ ?མ་', r'ཀུ་མ་'),
    (r'ཀུ་ ?ཤ', r'ཀུ་ཤ'),
    (r'ཀྲྀ ?ཥྞ་', r'ཀྲྀཥྞ་'),
    (r'པཎྜི་ ?ཏ་', r'པཎྜི་ཏ་'),
    (r'ཁ་ ?ཆེ', r'ཁ་ཆེ'),
    (r'དཱི་ ?པཾ་ ?ཀ་ ?ར་ ', r'དཱི་པཾ་ཀ་ར་ '),
    (r'ཤྲཱི་', r'ཤྲཱི་'),
    (r'ཛྙཱ་ ?ན་', r'ཛྙཱ་ན་'),
    (r'བཱ་ ?རཱ་ ?ཎ་ ?སཱི་', r'བཱ་རཱ་ཎ་སཱི་'),
    (r' བཻ་ ?ཌཱུརྻ་', r' བཻ་ཌཱུརྻ་'),
    (r'མཉྫུ', r'མཉྫུ'),
    (r'ཛེ་ ?ཏ', r'ཛེ་ཏ'),
    (r'ཛྙ་ ?དེ་ ?བ་', r'ཛྙ་དེ་བ་'),
    (r'ལུམྦཱི', r'ལུམྦཱི'),
    (r' རཀྵི་ ?ཏ', r' རཀྵི་ཏ'),
    (r'ཤྲདྡྷཱ་ ?ཀ་ ?ར་', r'ཤྲདྡྷཱ་ཀ་ར་'),
    (r'ཝརྨ་', r'ཝརྨ་'),
    (r'ལུམྦི', r'ལུམྦི'),
    (r'ལོ་ ?ཙཱ་ ?བ་', r'ལོ་ཙཱ་བ་'),
    (r'ཤཱ་ ?རི', r'ཤཱ་རི'),
    (r'ཤཱཀ་', r'ཤཱཀ་'),
    (r'ཤཱཀྱ', r'ཤཱཀྱ'),
    (r'ཤཱི་ ?ལེཎྡྲ་', r'ཤཱི་ལེཎྡྲ་'),
    (r'བོ་ ?དྷི་', r'བོ་དྷི་'),
    (r'ས་ ?ལེ་', r'ས་ལེ་'),
    (r'སཱ་ ?ལ', r'སཱ་ལ'),
    (r'སཱ་ ?ལུ་', r'སཱ་ལུ་'),
    (r'ཨ་ ?རུ་ ?ར', r'ཨ་རུ་ར'),
    (r'ནཻ་ ?རཉྫ་ ?ན་', r'ནཻ་རཉྫ་ན་'),
    (r'མ་ ?ཏི་ ?ཙི་ ?ཊ་', r'མ་ཏི་ཙི་ཊ་')
]


In [ ]:
#ར་ as a fusionned ladon is always separated, even in adverbial constructs
if word.endswith('ར') and word[:-1] not in lexicon:
	separate
    
#les རྫོགས་ཚིག་ toujours séparés (འོ་)
# mettre les particules finales non-ambigues
#… + དག་ separated except in exception list


In [10]:
test = 'དེ་ནས་ ཆོས་སྒྲ'
for rule in rules+ambiguous+propernouns:
    test = re.sub(rule[0], rule[1], test)

for i in corrections:
    test = test.replace(i[0], i[1])
print(test)

དེ་ ནས་ ཆོས་ སྒྲ


In [11]:
o = 0
for a in rules+ambiguous+propernouns:
    for b in corrections:
        if re.match(a[0], b[1]) or re.match(a[0], b[0]):
            print(a, '\n', b, '\n\n')
            o = o+1
print(o)

('(ཞེ|ཅེ)་ན', '\\1 ་ན') 
 ('ཅེ་ན', 'ཅེ་ ན') 


('གང་ དག', 'གང་དག') 
 ('གང་ དག་ མཉམ་པ ས་ དཔེ་ ཡིན་པ་ དེ་དག་ ནི་ མཉམ་པ འི་ དཔེའོ', 'གང་དག་ མཉམ་པ ས་ དཔེ་ ཡིན་པ་ དེ་དག་ ནི་ མཉམ་པ འི་ དཔེ འོ') 


('(གང་|ཅི་|རེ་|སུ་) ཞིག', '\\1ཞིག') 
 ('གང་ཞིག་ རྗེས་ སུ་ མི་མཐུན་པ', 'གང་ ཞིག་ རྗེས་ སུ་ མི་ མཐུན་པ') 


('(གང་|ཅི་|རེ་|སུ་) ཞིག', '\\1ཞིག') 
 ('གང་ཞིག་ པ', 'གང་ ཞིག་པ') 


('(གང་|གཞན་|དེ་)ཡང་', '\\1 ཡང་') 
 ('དེ་ཡང་ སྨྲ་ པོ', 'དེ་ ཡང་ སྨྲ་པོ') 


('ལ་ ?སོགས་ ?པ', 'ལ་སོགས་པ') 
 ('ལ་ སོགས་པའོ', 'ལ་སོགས་པ འོ') 


('([^་ ]+་)བཞིན', '\\1 བཞིན') 
 ('དེ་བཞིན་གཤེགས་པ འི་ སྐུ་ འདི་ནི', 'དེ་ བཞིན་ གཤེགས་པ འི་ སྐུ་ འདི་ ནི') 


('([^་ ]+་)བཞིན', '\\1 བཞིན') 
 ('དེ་བཞིན་ ལས་ བདག་ གི ར་ བྱ་བ', 'དེ་ བཞིན་ ལས་ བདག་ གི ར་ བྱ་ བ') 


('([^་ ]+་)བཞིན', '\\1 བཞིན') 
 ('དེ་བཞིན་ གཤེགས་པ་ ཐམས་ཅད་ ལ་ ཕྱག་འཚལ་', 'དེ་བཞིན་གཤེགས་པ་ ཐམས་ཅད་ ལ་ ཕྱག་ འཚལ་') 


('([^་ ]+་)བཞིན', '\\1 བཞིན') 
 ('ཡིད་བཞིན་ནོར་བུ་', 'ཡིད་ བཞིན་ ནོར་བུ་') 


('([^་ ]+་)བཞིན', '\\1 བཞིན') 
 ('རང་བཞིན་པས', 'རང་བཞིན་པ ས') 


('ཇི་ (ལྟ་བུ|ལྟར|སྙེད|ས